# 2. Model Exploration and Hyperparameter Tuning

## 📝 Overview
This notebook is the second step in the dementia prediction pipeline. Its purpose is to:
1.  **Load** the pre-cleaned and split data from `1_dataset_analysis.ipynb`.
2.  Define a **preprocessing pipeline** to handle scaling and encoding.
3.  Use **SMOTE** to address class imbalance in the training data.
4.  Train a variety of machine learning models using **GridSearchCV** to find the best hyperparameters for each.
5.  **Save** the trained models and their performance results for the final implementation phase.

## Libraries

In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
import warnings
warnings.filterwarnings('ignore')
import joblib
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFECV, RFE
from sklearn.decomposition import PCA
from sklearn.ensemble import  RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import label_binarize
from sklearn import metrics
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

## ML Model Results Storage Framework

In [8]:
# ML Model Result Storage Framework
precision = []
roc_auc = []

# Function to call for storing the results
def store_results(model, config, acc, f1_score, rec, prec, roc):
    """
    Store model performance results.
    """
    ML_Model.append(model)
    ML_Config.append(config)
    accuracy.append(round(acc, 6))
    f1.append(round(f1_score, 6))
    recall.append(round(rec, 6))
    precision.append(round(prec, 6))
    roc_auc.append(round(roc, 6))

# Function to display and save results
def display_and_save_results(filename_prefix='model_exploration'):
    """
    Create dataframe from results, display, and save to CSV in the 'AnalysisMain/results' directory.
    """
    # Creating the dataframe
    result = pd.DataFrame({
        'ML Model': ML_Model,
        'Configuration': ML_Config,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Recall': recall,
        'Precision': precision,
        'ROC_AUC': roc_auc,
    })
    
    # Remove duplicates if any
    result.drop_duplicates(subset=["ML Model", "Configuration"], inplace=True)
    
    print("\n" + "="*100)
    print("MODEL PERFORMANCE RESULTS")
    print("="*100)
    print(result.to_string(index=False))
    
    # Saving the result to a CSV file
    save_path = os.path.join(results_dir, f'{filename_prefix}_results.csv')
    result.to_csv(save_path, index=False)
    print(f"\nResults saved to {save_path}")
    
    # Sorting the dataframe on F1 Score and Accuracy
    sorted_result = result.sort_values(by=['F1 Score', 'Accuracy'], ascending=False).reset_index(drop=True)
    
    print("\n" + "="*100)
    print("SORTED MODEL PERFORMANCE RESULTS (by F1 Score and Accuracy)")
    print("="*100)
    print(sorted_result.to_string(index=False))
    
    # Saving the sorted result to a CSV file
    sorted_save_path = os.path.join(results_dir, f'sorted_{filename_prefix}_results.csv')
    sorted_result.to_csv(sorted_save_path, index=False)
    print(f"\nSorted results saved to {sorted_save_path}")
    
    return result, sorted_result

# Function to clear results
def clear_results():
    """Clear all stored results."""
    global ML_Model, ML_Config, accuracy, f1, recall, precision, roc_auc
    ML_Model.clear()
    ML_Config.clear()
    accuracy.clear()
    f1.clear()
    recall.clear()
    precision.clear()
    roc_auc.clear()
    print("Results cleared!")

# Function to plot model comparison
def plot_model_comparison(result_df, plot_filename="model_performance_comparison.png"):
    """
    Create visualization comparing model performances and save to 'AnalysisMain/plots'.
    """
    # Convert scores to percentages for plotting
    metrics_cols = ['Accuracy', 'F1 Score', 'Recall', 'Precision', 'ROC_AUC']
    plot_df = result_df.copy()
    
    for col in metrics_cols:
        plot_df[col] = plot_df[col] * 100
    
    # Create subplot for each metric
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    axes = axes.ravel()
    
    for idx, metric in enumerate(metrics_cols):
        # Group by model and get mean performance across configurations
        model_performance = plot_df.groupby('ML Model')[metric].mean().sort_values(ascending=False)
        
        # Create bar plot
        ax = axes[idx]
        bars = sns.barplot(x=model_performance.index, y=model_performance.values, ax=ax, palette='Blues_r')
        
        ax.set_title(f'Average {metric}', fontweight='bold')
        ax.set_ylabel(f'{metric} (%)')
        ax.set_xlabel('')
        ax.tick_params(axis='x', rotation=45)
        ax.grid(axis='y', alpha=0.5)
        
        # Add value labels on bars
        for bar in bars.patches:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=9)
    
    # Hide the last subplot if we have 5 metrics
    if len(metrics_cols) < 6:
        axes[5].set_visible(False)
    
    plt.suptitle('Model Performance Comparison', fontsize=20, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Save the plot
    save_path = os.path.join(plots_dir, plot_filename)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Comparison plot saved to: {save_path}")

print("Model results storage framework loaded successfully!")
print("This framework will save results, plots, and models to the 'ModelExploration' directory.")

Model results storage framework loaded successfully!
This framework will save results, plots, and models to the 'ModelExploration' directory.


# Define Preprocessing Pipeline

Before training the models, we need to create a preprocessing pipeline. This pipeline will handle:
1.  **Scaling**: Applying `StandardScaler` to all numerical features to standardize their range.
2.  **Encoding**: Applying `OneHotEncoder` to the categorical feature (`M/F`) to convert it into a numerical format.

We use a `ColumnTransformer` to apply these different transformations to the correct columns. This ensures that the same steps are consistently applied during both training and validation.

In [9]:
# Define Preprocessing Pipeline

# Define the directory where the processed data was saved from the previous notebook
processed_data_dir = 'Analysis/processed_data'

# Load the training and validation sets
X_train = joblib.load(os.path.join(processed_data_dir, 'X_train.joblib'))
X_val = joblib.load(os.path.join(processed_data_dir, 'X_val.joblib'))
y_train = joblib.load(os.path.join(processed_data_dir, 'y_train.joblib'))
y_val = joblib.load(os.path.join(processed_data_dir, 'y_val.joblib'))

print("Data loaded successfully from 'processed_data' directory!")
print("-" * 50)
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"\nTraining target distribution:\n{y_train.value_counts(normalize=True)}")
print(f"\nValidation target distribution:\n{y_val.value_counts(normalize=True)}")

# Display the first few rows of the training data to confirm
print("\nFirst 5 rows of X_train:")
X_train.head()

Data loaded successfully from 'processed_data' directory!
--------------------------------------------------
X_train shape: (647, 12)
X_val shape: (162, 12)

Training target distribution:
Group
Nondemented   0.774343
Demented      0.225657
Name: proportion, dtype: float64

Validation target distribution:
Group
Nondemented   0.771605
Demented      0.228395
Name: proportion, dtype: float64

First 5 rows of X_train:


,M/F,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay,Visit,MR Delay
764,M,84,14.000000,2.000000,22.000000,0.500000,1550,0.665000,1.132000,2.000000,2.000000,621.000000
213,M,75,5.000000,2.000000,29.000000,0.000000,1534,0.771000,1.144000,2.000000,1.000000,0.000000
382,F,69,4.000000,3.000000,29.000000,0.000000,1380,0.809000,1.272000,2.000000,1.000000,0.000000
456,F,80,16.000000,2.000000,29.000000,0.000000,1323,0.738000,1.326000,2.000000,2.000000,730.000000
393,F,50,12.000000,2.000000,30.000000,0.000000,1385,0.819000,1.267000,2.000000,1.000000,0.000000


---

# SVM

### SVM with PCA 90

In [ ]:

# =============================================================================
# SVM with Configuration-Specific Hyperparameter Grids
# =============================================================================
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE, RFECV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn import metrics
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.combine import SMOTETomek
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Define preprocessor for categorical and numeric features
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# Create preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

# Apply preprocessing to get fully numeric data first
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

# Get feature names after preprocessing
try:
    feature_names = (numeric_features + 
                    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
except:
    feature_names = [f'feature_{i}' for i in range(X_train_preprocessed.shape[1])]

X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=feature_names, index=X_train.index)
X_val_preprocessed = pd.DataFrame(X_val_preprocessed, columns=feature_names, index=X_val.index)

print(f"\nPreprocessed data shape: {X_train_preprocessed.shape}")
print(f"All features are now numeric: {X_train_preprocessed.select_dtypes(include=np.number).shape[1] == X_train_preprocessed.shape[1]}")

# Clear previous results
try:
    clear_results()
except:
    ML_Model = []
    ML_Config = []
    accuracy = []
    f1 = []
    recall = []
    precision = []
    roc_auc = []
    print("Initialized result storage lists.")

print("\n=== START: SVM Configuration Sweep with Custom Hyperparameters ===\n")

# =============================================================================
# Configuration-Specific Hyperparameter Grids
# =============================================================================

# Grid 1: Preprocessed Data - Focus on RBF and linear kernels
param_grid_1 = {
    'C': [0.01, 0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01],
    'kernel': ['rbf', 'linear'],
    'degree': [2],  # Not used for rbf/linear but required
    'coef0': [0.0]
}

# Grid 2: Normalized Data - Explore polynomial kernels
param_grid_2 = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1],
    'kernel': ['poly', 'rbf'],
    'degree': [2, 3, 4],
    'coef0': [0.0, 0.5, 1.0]
}

# Grid 3: SelectKBest - Focus on simpler models
param_grid_3 = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['rbf', 'poly', 'linear'],
    'degree': [2, 3],
    'coef0': [0.0, 0.1]
}

# Grid 4: RFECV - Similar to SelectKBest but different ranges
param_grid_4 = {
    'C': [0.5, 1, 5, 10],
    'gamma': ['scale', 0.001, 0.01],
    'kernel': ['rbf', 'sigmoid'],
    'degree': [2, 3],
    'coef0': [0.0, 0.5]
}

# Grid 5: PCA - Focus on linear and sigmoid
param_grid_5 = {
    'C': [0.1, 1, 10, 50],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['linear', 'sigmoid', 'rbf'],
    'degree': [2],
    'coef0': [0.0, 0.5, 1.0]
}

# Grid 6 & 7: SMOTE pipelines - Balanced approach (Oversampling)
param_grid_smote = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 0.001, 0.01, 0.1],
    'model__kernel': ['rbf', 'poly', 'linear'],
    'model__degree': [2, 3],
    'model__coef0': [0.0, 0.5]
}

# Grid 8-11: Undersampling and Combined pipelines
param_grid_sampling = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 0.001, 0.01, 0.1],
    'model__kernel': ['rbf', 'poly', 'linear'],
    'model__degree': [2, 3],
    'model__coef0': [0.0, 0.5]
}

# Map grids to configurations
hyperparameter_grids = {
    'Preprocessed Data': param_grid_1,
    'Normalized Data': param_grid_2,
    'SelectKBest': param_grid_3,
    'RFECV': param_grid_4,
    'PCA': param_grid_5,
    'SMOTE + StandardScaler': param_grid_smote,
    'SMOTE + GridSearchCV': param_grid_smote,
    'RandomUnderSampler': param_grid_sampling,
    'TomekLinks': param_grid_sampling,
    'NearMiss': param_grid_sampling,
    'SMOTE + Tomek': param_grid_sampling,
}

# Initialize configurations list
configurations = []

# --- Configuration 1: Preprocessed Data ---
configurations.append(('Preprocessed Data', 'array', X_train_preprocessed, X_val_preprocessed))
print("✓ Configuration 1: Preprocessed Data")

# --- Configuration 2: Normalized Data (MinMax on preprocessed) ---
scaler_minmax = MinMaxScaler()
X_train_normalized = pd.DataFrame(
    scaler_minmax.fit_transform(X_train_preprocessed), 
    columns=X_train_preprocessed.columns, 
    index=X_train_preprocessed.index
)
X_val_normalized = pd.DataFrame(
    scaler_minmax.transform(X_val_preprocessed), 
    columns=X_val_preprocessed.columns, 
    index=X_val_preprocessed.index
)
configurations.append(('Normalized Data', 'array', X_train_normalized, X_val_normalized))
print("✓ Configuration 2: Normalized Data (MinMax)")

# --- Configuration 3: SelectKBest ---
print("\n=== SelectKBest Feature Selection ===")
scores = []
max_features = min(X_train_normalized.shape[1], 20)

for k in range(1, max_features + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_tr_k = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(
        SVC(kernel='linear', random_state=RANDOM_STATE), 
        X_tr_k, y_train, cv=5, scoring='accuracy', n_jobs=-1
    ).mean()
    scores.append(score)

optimal_k = int(np.argmax(scores) + 1)
print(f"Optimal number of features: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = pd.DataFrame(
    kbest.fit_transform(X_train_normalized, y_train), 
    columns=X_train_normalized.columns[kbest.get_support()]
)
X_val_kbest = pd.DataFrame(
    kbest.transform(X_val_normalized), 
    columns=X_train_kbest.columns
)
configurations.append(('SelectKBest', 'array', X_train_kbest, X_val_kbest))
print("✓ Configuration 3: SelectKBest")

# --- Configuration 4: RFECV ---
print("\n=== RFECV Feature Selection ===")
svm_estimator = SVC(kernel='linear', random_state=RANDOM_STATE)
rfecv = RFECV(
    estimator=svm_estimator, 
    step=1, 
    cv=StratifiedKFold(5), 
    scoring='accuracy', 
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=svm_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = pd.DataFrame(
    rfe.fit_transform(X_train_kbest, y_train), 
    columns=X_train_kbest.columns[rfe.get_support()]
)
X_val_rfe = pd.DataFrame(
    rfe.transform(X_val_kbest), 
    columns=X_train_rfe.columns
)
configurations.append(('RFECV', 'array', X_train_rfe, X_val_rfe))
print("✓ Configuration 4: RFECV")

# --- Configuration 5: PCA ---
print("\n=== PCA Dimensionality Reduction ===")
pca_full = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
desired_variance = 0.90
n_components = int(np.argmax(cumulative_variance >= desired_variance) + 1)
n_components = max(2, n_components)
print(f'Number of components for {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components, random_state=RANDOM_STATE)
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_rfe), index=X_train_rfe.index)
X_val_pca = pd.DataFrame(pca.transform(X_val_rfe), index=X_val_rfe.index)
configurations.append(('PCA', 'array', X_train_pca, X_val_pca))
print("✓ Configuration 5: PCA")

# --- Configuration 6: SMOTE + StandardScaler (Pipeline) ---
pipeline_smote_scaler = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + StandardScaler', 'pipeline', pipeline_smote_scaler, None))
print("✓ Configuration 6: SMOTE + StandardScaler (Pipeline)")

# --- Configuration 7: SMOTE + GridSearchCV (Pipeline) ---
pipeline_smote_grid = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + GridSearchCV', 'pipeline', pipeline_smote_grid, None))
print("✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)")

# =============================================================================
# UNDERSAMPLING CONFIGURATIONS
# =============================================================================
print("\n=== Adding Undersampling Configurations ===")

# --- Configuration 8: RandomUnderSampler ---
pipeline_rus = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', RandomUnderSampler(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('RandomUnderSampler', 'pipeline', pipeline_rus, None))
print("✓ Configuration 8: RandomUnderSampler (Undersampling)")

# --- Configuration 9: TomekLinks ---
pipeline_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', TomekLinks()),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('TomekLinks', 'pipeline', pipeline_tomek, None))
print("✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)")

# --- Configuration 10: NearMiss ---
pipeline_nearmiss = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', NearMiss(version=1)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('NearMiss', 'pipeline', pipeline_nearmiss, None))
print("✓ Configuration 10: NearMiss (Undersampling - selective)")

# --- Configuration 11: SMOTE + Tomek (Combined) ---
pipeline_smote_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('sampler', SMOTETomek(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + Tomek', 'pipeline', pipeline_smote_tomek, None))
print("✓ Configuration 11: SMOTE + Tomek (Combined Over + Under)")

print(f"\nTotal configurations: {len(configurations)}")

# Safe ROC AUC helper
def safe_roc_auc(y_true, y_proba):
    try:
        if isinstance(y_proba, np.ndarray) and y_proba.shape[1] == 2:
            return metrics.roc_auc_score(y_true, y_proba[:, 1])
        else:
            return metrics.roc_auc_score(
                pd.get_dummies(y_true), y_proba, 
                multi_class='ovr', average='macro'
            )
    except Exception:
        return np.nan

# =============================================================================
# Run SVM with Configuration-Specific GridSearchCV
# =============================================================================
print("\n" + "="*80)
print("RUNNING SVM WITH CONFIGURATION-SPECIFIC HYPERPARAMETER TUNING")
print("="*80)

for name, kind, X_tr_cfg, X_val_cfg in configurations:
    print(f"\n{'='*80}")
    print(f"Configuration: {name}")
    print(f"{'='*80}")
    
    # Get the specific parameter grid for this configuration
    param_grid = hyperparameter_grids[name]
    
    print(f"Hyperparameter grid for '{name}':")
    for key, values in param_grid.items():
        print(f"  {key}: {values}")
    print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")
    
    try:
        if kind == 'pipeline':
            pipeline = X_tr_cfg
            grid_search = GridSearchCV(
                pipeline, param_grid, 
                cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_train)
            y_val_pred = best_model.predict(X_val)
            y_train_proba = best_model.predict_proba(X_train)
            y_val_proba = best_model.predict_proba(X_val)
        else:
            grid_search = GridSearchCV(
                SVC(probability=True, random_state=RANDOM_STATE), 
                param_grid, cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_tr_cfg, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_tr_cfg)
            y_val_pred = best_model.predict(X_val_cfg)
            y_train_proba = best_model.predict_proba(X_tr_cfg)
            y_val_proba = best_model.predict_proba(X_val_cfg)

        # Calculate train-test gap for overfitting detection
        train_acc = metrics.accuracy_score(y_train, y_train_pred)
        test_acc = metrics.accuracy_score(y_val, y_val_pred)
        train_test_gap = train_acc - test_acc

        # Build metrics dictionary
        metrics_dict = {
            "Dataset": ["Training", "Test"],
            "Accuracy": [train_acc, test_acc],
            "F1 Score": [
                metrics.f1_score(y_train, y_train_pred, average='macro'),
                metrics.f1_score(y_val, y_val_pred, average='macro'),
            ],
            "Recall": [
                metrics.recall_score(y_train, y_train_pred, average='macro'),
                metrics.recall_score(y_val, y_val_pred, average='macro'),
            ],
            "Precision": [
                metrics.precision_score(y_train, y_train_pred, average='macro', zero_division=0),
                metrics.precision_score(y_val, y_val_pred, average='macro', zero_division=0),
            ],
            "AUC-ROC": [
                safe_roc_auc(y_train, y_train_proba),
                safe_roc_auc(y_val, y_val_proba),
            ]
        }

        df_metrics = pd.DataFrame(metrics_dict)
        pd.options.display.float_format = '{:.6f}'.format
        print("\n📊 Support Vector Machine Model Performance Metrics")
        print(df_metrics.to_string(index=False))

        # Overfitting warning
        if train_test_gap > 0.10:
            print(f"\n⚠️  WARNING: Overfitting detected! Train-Test gap: {train_test_gap:.4f}")
        elif train_test_gap < 0.05:
            print(f"\n✓ Good generalization. Train-Test gap: {train_test_gap:.4f}")
        else:
            print(f"\n→ Acceptable gap: {train_test_gap:.4f}")

        # Store test metrics
        test_metrics = df_metrics[df_metrics['Dataset'] == 'Test'].iloc[0]
        try:
            store_results(
                'SVM',
                name,
                float(test_metrics['Accuracy']),
                float(test_metrics['F1 Score']),
                float(test_metrics['Recall']),
                float(test_metrics['Precision']),
                float(test_metrics['AUC-ROC'])
            )
        except:
            ML_Model.append('SVM')
            ML_Config.append(name)
            accuracy.append(round(float(test_metrics['Accuracy']), 6))
            f1.append(round(float(test_metrics['F1 Score']), 6))
            recall.append(round(float(test_metrics['Recall']), 6))
            precision.append(round(float(test_metrics['Precision']), 6))
            roc_auc.append(round(float(test_metrics['AUC-ROC']), 6))

        print("\n🎯 Best hyperparameters found:")
        best_params = grid_search.best_params_
        for param, value in best_params.items():
            print(f"  {param}: {value}")
        print(f"\nBest CV score: {grid_search.best_score_:.6f}")
        
    except Exception as e:
        print(f"❌ Error in configuration '{name}': {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*80)
print("✅ SVM evaluation complete for all configurations.")
print("="*80)

# Display final results
try:
    display_and_save_results('svm_all_configs')
except:
    result = pd.DataFrame({
        'ML Model': ML_Model,
        'Configuration': ML_Config,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Recall': recall,
        'Precision': precision,
        'ROC_AUC': roc_auc,
    })
    print("\n📈 Final Results:")
    print(result.to_string(index=False))
    
    # Sort by F1 Score
    sorted_result = result.sort_values(by=['F1 Score', 'Accuracy'], ascending=False).reset_index(drop=True)
    print("\n🏆 Sorted Results (by F1 Score):")
    print(sorted_result.to_string(index=False))

    # Group by sampling technique
    print("\n📊 Performance by Sampling Technique:")
    print("-" * 80)
    sampling_groups = {
        'No Sampling': ['Preprocessed Data', 'Normalized Data', 'SelectKBest', 'RFECV', 'PCA'],
        'Oversampling': ['SMOTE + StandardScaler', 'SMOTE + GridSearchCV'],
        'Undersampling': ['RandomUnderSampler', 'TomekLinks', 'NearMiss'],
        'Combined': ['SMOTE + Tomek']
    }

    for group_name, configs in sampling_groups.items():
        group_data = result[result['Configuration'].isin(configs)]
        if not group_data.empty:
            print(f"\n{group_name}:")
            print(f"  Avg Accuracy: {group_data['Accuracy'].mean():.6f}")
            print(f"  Avg F1 Score: {group_data['F1 Score'].mean():.6f}")
            print(f"  Avg ROC-AUC: {group_data['ROC_AUC'].mean():.6f}")
            print(f"  Best Config: {group_data.loc[group_data['F1 Score'].idxmax(), 'Configuration']}")



Numeric features: ['Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'Delay', 'Visit', 'MR Delay']
Categorical features: ['M/F']

Preprocessed data shape: (647, 13)
All features are now numeric: True
Initialized result storage lists.

=== START: SVM Configuration Sweep with Custom Hyperparameters ===

✓ Configuration 1: Preprocessed Data
✓ Configuration 2: Normalized Data (MinMax)

=== SelectKBest Feature Selection ===
Optimal number of features: 10
✓ Configuration 3: SelectKBest

=== RFECV Feature Selection ===
Optimal number of features by RFECV: 3
✓ Configuration 4: RFECV

=== PCA Dimensionality Reduction ===
Number of components for 90.0% variance: 3
✓ Configuration 5: PCA
✓ Configuration 6: SMOTE + StandardScaler (Pipeline)
✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)

=== Adding Undersampling Configurations ===
✓ Configuration 8: RandomUnderSampler (Undersampling)
✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)
✓ Configuration 10: NearMiss (U

### SVM with PCA 95

In [5]:

# =============================================================================
# SVM with Configuration-Specific Hyperparameter Grids
# =============================================================================
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE, RFECV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn import metrics
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.combine import SMOTETomek
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Define preprocessor for categorical and numeric features
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# Create preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

# Apply preprocessing to get fully numeric data first
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

# Get feature names after preprocessing
try:
    feature_names = (numeric_features + 
                    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
except:
    feature_names = [f'feature_{i}' for i in range(X_train_preprocessed.shape[1])]

X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=feature_names, index=X_train.index)
X_val_preprocessed = pd.DataFrame(X_val_preprocessed, columns=feature_names, index=X_val.index)

print(f"\nPreprocessed data shape: {X_train_preprocessed.shape}")
print(f"All features are now numeric: {X_train_preprocessed.select_dtypes(include=np.number).shape[1] == X_train_preprocessed.shape[1]}")

# Clear previous results
try:
    clear_results()
except:
    ML_Model = []
    ML_Config = []
    accuracy = []
    f1 = []
    recall = []
    precision = []
    roc_auc = []
    print("Initialized result storage lists.")

print("\n=== START: SVM Configuration Sweep with Custom Hyperparameters ===\n")

# =============================================================================
# Configuration-Specific Hyperparameter Grids
# =============================================================================

# Grid 1: Preprocessed Data - Focus on RBF and linear kernels
param_grid_1 = {
    'C': [0.01, 0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01],
    'kernel': ['rbf', 'linear'],
    'degree': [2],  # Not used for rbf/linear but required
    'coef0': [0.0]
}

# Grid 2: Normalized Data - Explore polynomial kernels
param_grid_2 = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1],
    'kernel': ['poly', 'rbf'],
    'degree': [2, 3, 4],
    'coef0': [0.0, 0.5, 1.0]
}

# Grid 3: SelectKBest - Focus on simpler models
param_grid_3 = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['rbf', 'poly', 'linear'],
    'degree': [2, 3],
    'coef0': [0.0, 0.1]
}

# Grid 4: RFECV - Similar to SelectKBest but different ranges
param_grid_4 = {
    'C': [0.5, 1, 5, 10],
    'gamma': ['scale', 0.001, 0.01],
    'kernel': ['rbf', 'sigmoid'],
    'degree': [2, 3],
    'coef0': [0.0, 0.5]
}

# Grid 5: PCA - Focus on linear and sigmoid
param_grid_5 = {
    'C': [0.1, 1, 10, 50],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['linear', 'sigmoid', 'rbf'],
    'degree': [2],
    'coef0': [0.0, 0.5, 1.0]
}

# Grid 6 & 7: SMOTE pipelines - Balanced approach (Oversampling)
param_grid_smote = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 0.001, 0.01, 0.1],
    'model__kernel': ['rbf', 'poly', 'linear'],
    'model__degree': [2, 3],
    'model__coef0': [0.0, 0.5]
}

# Grid 8-11: Undersampling and Combined pipelines
param_grid_sampling = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 0.001, 0.01, 0.1],
    'model__kernel': ['rbf', 'poly', 'linear'],
    'model__degree': [2, 3],
    'model__coef0': [0.0, 0.5]
}

# Map grids to configurations
hyperparameter_grids = {
    'Preprocessed Data': param_grid_1,
    'Normalized Data': param_grid_2,
    'SelectKBest': param_grid_3,
    'RFECV': param_grid_4,
    'PCA': param_grid_5,
    'SMOTE + StandardScaler': param_grid_smote,
    'SMOTE + GridSearchCV': param_grid_smote,
    'RandomUnderSampler': param_grid_sampling,
    'TomekLinks': param_grid_sampling,
    'NearMiss': param_grid_sampling,
    'SMOTE + Tomek': param_grid_sampling,
}

# Initialize configurations list
configurations = []

# --- Configuration 1: Preprocessed Data ---
configurations.append(('Preprocessed Data', 'array', X_train_preprocessed, X_val_preprocessed))
print("✓ Configuration 1: Preprocessed Data")

# --- Configuration 2: Normalized Data (MinMax on preprocessed) ---
scaler_minmax = MinMaxScaler()
X_train_normalized = pd.DataFrame(
    scaler_minmax.fit_transform(X_train_preprocessed), 
    columns=X_train_preprocessed.columns, 
    index=X_train_preprocessed.index
)
X_val_normalized = pd.DataFrame(
    scaler_minmax.transform(X_val_preprocessed), 
    columns=X_val_preprocessed.columns, 
    index=X_val_preprocessed.index
)
configurations.append(('Normalized Data', 'array', X_train_normalized, X_val_normalized))
print("✓ Configuration 2: Normalized Data (MinMax)")

# --- Configuration 3: SelectKBest ---
print("\n=== SelectKBest Feature Selection ===")
scores = []
max_features = min(X_train_normalized.shape[1], 20)

for k in range(1, max_features + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_tr_k = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(
        SVC(kernel='linear', random_state=RANDOM_STATE), 
        X_tr_k, y_train, cv=5, scoring='accuracy', n_jobs=-1
    ).mean()
    scores.append(score)

optimal_k = int(np.argmax(scores) + 1)
print(f"Optimal number of features: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = pd.DataFrame(
    kbest.fit_transform(X_train_normalized, y_train), 
    columns=X_train_normalized.columns[kbest.get_support()]
)
X_val_kbest = pd.DataFrame(
    kbest.transform(X_val_normalized), 
    columns=X_train_kbest.columns
)
configurations.append(('SelectKBest', 'array', X_train_kbest, X_val_kbest))
print("✓ Configuration 3: SelectKBest")

# --- Configuration 4: RFECV ---
print("\n=== RFECV Feature Selection ===")
svm_estimator = SVC(kernel='linear', random_state=RANDOM_STATE)
rfecv = RFECV(
    estimator=svm_estimator, 
    step=1, 
    cv=StratifiedKFold(5), 
    scoring='accuracy', 
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=svm_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = pd.DataFrame(
    rfe.fit_transform(X_train_kbest, y_train), 
    columns=X_train_kbest.columns[rfe.get_support()]
)
X_val_rfe = pd.DataFrame(
    rfe.transform(X_val_kbest), 
    columns=X_train_rfe.columns
)
configurations.append(('RFECV', 'array', X_train_rfe, X_val_rfe))
print("✓ Configuration 4: RFECV")

# --- Configuration 5: PCA ---
print("\n=== PCA Dimensionality Reduction ===")
pca_full = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
desired_variance = 0.95
n_components = int(np.argmax(cumulative_variance >= desired_variance) + 1)
n_components = max(2, n_components)
print(f'Number of components for {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components, random_state=RANDOM_STATE)
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_rfe), index=X_train_rfe.index)
X_val_pca = pd.DataFrame(pca.transform(X_val_rfe), index=X_val_rfe.index)
configurations.append(('PCA', 'array', X_train_pca, X_val_pca))
print("✓ Configuration 5: PCA")

# --- Configuration 6: SMOTE + StandardScaler (Pipeline) ---
pipeline_smote_scaler = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + StandardScaler', 'pipeline', pipeline_smote_scaler, None))
print("✓ Configuration 6: SMOTE + StandardScaler (Pipeline)")

# --- Configuration 7: SMOTE + GridSearchCV (Pipeline) ---
pipeline_smote_grid = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + GridSearchCV', 'pipeline', pipeline_smote_grid, None))
print("✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)")

# =============================================================================
# UNDERSAMPLING CONFIGURATIONS
# =============================================================================
print("\n=== Adding Undersampling Configurations ===")

# --- Configuration 8: RandomUnderSampler ---
pipeline_rus = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', RandomUnderSampler(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('RandomUnderSampler', 'pipeline', pipeline_rus, None))
print("✓ Configuration 8: RandomUnderSampler (Undersampling)")

# --- Configuration 9: TomekLinks ---
pipeline_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', TomekLinks()),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('TomekLinks', 'pipeline', pipeline_tomek, None))
print("✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)")

# --- Configuration 10: NearMiss ---
pipeline_nearmiss = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', NearMiss(version=1)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('NearMiss', 'pipeline', pipeline_nearmiss, None))
print("✓ Configuration 10: NearMiss (Undersampling - selective)")

# --- Configuration 11: SMOTE + Tomek (Combined) ---
pipeline_smote_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('sampler', SMOTETomek(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + Tomek', 'pipeline', pipeline_smote_tomek, None))
print("✓ Configuration 11: SMOTE + Tomek (Combined Over + Under)")

print(f"\nTotal configurations: {len(configurations)}")

# Safe ROC AUC helper
def safe_roc_auc(y_true, y_proba):
    try:
        if isinstance(y_proba, np.ndarray) and y_proba.shape[1] == 2:
            return metrics.roc_auc_score(y_true, y_proba[:, 1])
        else:
            return metrics.roc_auc_score(
                pd.get_dummies(y_true), y_proba, 
                multi_class='ovr', average='macro'
            )
    except Exception:
        return np.nan

# =============================================================================
# Run SVM with Configuration-Specific GridSearchCV
# =============================================================================
print("\n" + "="*80)
print("RUNNING SVM WITH CONFIGURATION-SPECIFIC HYPERPARAMETER TUNING")
print("="*80)

for name, kind, X_tr_cfg, X_val_cfg in configurations:
    print(f"\n{'='*80}")
    print(f"Configuration: {name}")
    print(f"{'='*80}")
    
    # Get the specific parameter grid for this configuration
    param_grid = hyperparameter_grids[name]
    
    print(f"Hyperparameter grid for '{name}':")
    for key, values in param_grid.items():
        print(f"  {key}: {values}")
    print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")
    
    try:
        if kind == 'pipeline':
            pipeline = X_tr_cfg
            grid_search = GridSearchCV(
                pipeline, param_grid, 
                cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_train)
            y_val_pred = best_model.predict(X_val)
            y_train_proba = best_model.predict_proba(X_train)
            y_val_proba = best_model.predict_proba(X_val)
        else:
            grid_search = GridSearchCV(
                SVC(probability=True, random_state=RANDOM_STATE), 
                param_grid, cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_tr_cfg, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_tr_cfg)
            y_val_pred = best_model.predict(X_val_cfg)
            y_train_proba = best_model.predict_proba(X_tr_cfg)
            y_val_proba = best_model.predict_proba(X_val_cfg)

        # Calculate train-test gap for overfitting detection
        train_acc = metrics.accuracy_score(y_train, y_train_pred)
        test_acc = metrics.accuracy_score(y_val, y_val_pred)
        train_test_gap = train_acc - test_acc

        # Build metrics dictionary
        metrics_dict = {
            "Dataset": ["Training", "Test"],
            "Accuracy": [train_acc, test_acc],
            "F1 Score": [
                metrics.f1_score(y_train, y_train_pred, average='macro'),
                metrics.f1_score(y_val, y_val_pred, average='macro'),
            ],
            "Recall": [
                metrics.recall_score(y_train, y_train_pred, average='macro'),
                metrics.recall_score(y_val, y_val_pred, average='macro'),
            ],
            "Precision": [
                metrics.precision_score(y_train, y_train_pred, average='macro', zero_division=0),
                metrics.precision_score(y_val, y_val_pred, average='macro', zero_division=0),
            ],
            "AUC-ROC": [
                safe_roc_auc(y_train, y_train_proba),
                safe_roc_auc(y_val, y_val_proba),
            ]
        }

        df_metrics = pd.DataFrame(metrics_dict)
        pd.options.display.float_format = '{:.6f}'.format
        print("\n📊 Support Vector Machine Model Performance Metrics")
        print(df_metrics.to_string(index=False))

        # Overfitting warning
        if train_test_gap > 0.10:
            print(f"\n⚠️  WARNING: Overfitting detected! Train-Test gap: {train_test_gap:.4f}")
        elif train_test_gap < 0.05:
            print(f"\n✓ Good generalization. Train-Test gap: {train_test_gap:.4f}")
        else:
            print(f"\n→ Acceptable gap: {train_test_gap:.4f}")

        # Store test metrics
        test_metrics = df_metrics[df_metrics['Dataset'] == 'Test'].iloc[0]
        try:
            store_results(
                'SVM',
                name,
                float(test_metrics['Accuracy']),
                float(test_metrics['F1 Score']),
                float(test_metrics['Recall']),
                float(test_metrics['Precision']),
                float(test_metrics['AUC-ROC'])
            )
        except:
            ML_Model.append('SVM')
            ML_Config.append(name)
            accuracy.append(round(float(test_metrics['Accuracy']), 6))
            f1.append(round(float(test_metrics['F1 Score']), 6))
            recall.append(round(float(test_metrics['Recall']), 6))
            precision.append(round(float(test_metrics['Precision']), 6))
            roc_auc.append(round(float(test_metrics['AUC-ROC']), 6))

        print("\n🎯 Best hyperparameters found:")
        best_params = grid_search.best_params_
        for param, value in best_params.items():
            print(f"  {param}: {value}")
        print(f"\nBest CV score: {grid_search.best_score_:.6f}")
        
    except Exception as e:
        print(f"❌ Error in configuration '{name}': {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*80)
print("✅ SVM evaluation complete for all configurations.")
print("="*80)

# Display final results
try:
    display_and_save_results('svm_all_configs')
except:
    result = pd.DataFrame({
        'ML Model': ML_Model,
        'Configuration': ML_Config,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Recall': recall,
        'Precision': precision,
        'ROC_AUC': roc_auc,
    })
    print("\n📈 Final Results:")
    print(result.to_string(index=False))
    
    # Sort by F1 Score
    sorted_result = result.sort_values(by=['F1 Score', 'Accuracy'], ascending=False).reset_index(drop=True)
    print("\n🏆 Sorted Results (by F1 Score):")
    print(sorted_result.to_string(index=False))

    # Group by sampling technique
    print("\n📊 Performance by Sampling Technique:")
    print("-" * 80)
    sampling_groups = {
        'No Sampling': ['Preprocessed Data', 'Normalized Data', 'SelectKBest', 'RFECV', 'PCA'],
        'Oversampling': ['SMOTE + StandardScaler', 'SMOTE + GridSearchCV'],
        'Undersampling': ['RandomUnderSampler', 'TomekLinks', 'NearMiss'],
        'Combined': ['SMOTE + Tomek']
    }

    for group_name, configs in sampling_groups.items():
        group_data = result[result['Configuration'].isin(configs)]
        if not group_data.empty:
            print(f"\n{group_name}:")
            print(f"  Avg Accuracy: {group_data['Accuracy'].mean():.6f}")
            print(f"  Avg F1 Score: {group_data['F1 Score'].mean():.6f}")
            print(f"  Avg ROC-AUC: {group_data['ROC_AUC'].mean():.6f}")
            print(f"  Best Config: {group_data.loc[group_data['F1 Score'].idxmax(), 'Configuration']}")



Numeric features: ['Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'Delay', 'Visit', 'MR Delay']
Categorical features: ['M/F']

Preprocessed data shape: (647, 13)
All features are now numeric: True
Results cleared!

=== START: SVM Configuration Sweep with Custom Hyperparameters ===

✓ Configuration 1: Preprocessed Data
✓ Configuration 2: Normalized Data (MinMax)

=== SelectKBest Feature Selection ===
Optimal number of features: 10
✓ Configuration 3: SelectKBest

=== RFECV Feature Selection ===
Optimal number of features by RFECV: 3
✓ Configuration 4: RFECV

=== PCA Dimensionality Reduction ===
Number of components for 95.0% variance: 3
✓ Configuration 5: PCA
✓ Configuration 6: SMOTE + StandardScaler (Pipeline)
✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)

=== Adding Undersampling Configurations ===
✓ Configuration 8: RandomUnderSampler (Undersampling)
✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)
✓ Configuration 10: NearMiss (Undersampling - se

### SVM with PCA 99

In [6]:

# =============================================================================
# SVM with Configuration-Specific Hyperparameter Grids
# =============================================================================
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE, RFECV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn import metrics
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.combine import SMOTETomek
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Define preprocessor for categorical and numeric features
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# Create preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

# Apply preprocessing to get fully numeric data first
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

# Get feature names after preprocessing
try:
    feature_names = (numeric_features + 
                    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
except:
    feature_names = [f'feature_{i}' for i in range(X_train_preprocessed.shape[1])]

X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=feature_names, index=X_train.index)
X_val_preprocessed = pd.DataFrame(X_val_preprocessed, columns=feature_names, index=X_val.index)

print(f"\nPreprocessed data shape: {X_train_preprocessed.shape}")
print(f"All features are now numeric: {X_train_preprocessed.select_dtypes(include=np.number).shape[1] == X_train_preprocessed.shape[1]}")

# Clear previous results
try:
    clear_results()
except:
    ML_Model = []
    ML_Config = []
    accuracy = []
    f1 = []
    recall = []
    precision = []
    roc_auc = []
    print("Initialized result storage lists.")

print("\n=== START: SVM Configuration Sweep with Custom Hyperparameters ===\n")

# =============================================================================
# Configuration-Specific Hyperparameter Grids
# =============================================================================

# Grid 1: Preprocessed Data - Focus on RBF and linear kernels
param_grid_1 = {
    'C': [0.01, 0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01],
    'kernel': ['rbf', 'linear'],
    'degree': [2],  # Not used for rbf/linear but required
    'coef0': [0.0]
}

# Grid 2: Normalized Data - Explore polynomial kernels
param_grid_2 = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1],
    'kernel': ['poly', 'rbf'],
    'degree': [2, 3, 4],
    'coef0': [0.0, 0.5, 1.0]
}

# Grid 3: SelectKBest - Focus on simpler models
param_grid_3 = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['rbf', 'poly', 'linear'],
    'degree': [2, 3],
    'coef0': [0.0, 0.1]
}

# Grid 4: RFECV - Similar to SelectKBest but different ranges
param_grid_4 = {
    'C': [0.5, 1, 5, 10],
    'gamma': ['scale', 0.001, 0.01],
    'kernel': ['rbf', 'sigmoid'],
    'degree': [2, 3],
    'coef0': [0.0, 0.5]
}

# Grid 5: PCA - Focus on linear and sigmoid
param_grid_5 = {
    'C': [0.1, 1, 10, 50],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['linear', 'sigmoid', 'rbf'],
    'degree': [2],
    'coef0': [0.0, 0.5, 1.0]
}

# Grid 6 & 7: SMOTE pipelines - Balanced approach (Oversampling)
param_grid_smote = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 0.001, 0.01, 0.1],
    'model__kernel': ['rbf', 'poly', 'linear'],
    'model__degree': [2, 3],
    'model__coef0': [0.0, 0.5]
}

# Grid 8-11: Undersampling and Combined pipelines
param_grid_sampling = {
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': ['scale', 0.001, 0.01, 0.1],
    'model__kernel': ['rbf', 'poly', 'linear'],
    'model__degree': [2, 3],
    'model__coef0': [0.0, 0.5]
}

# Map grids to configurations
hyperparameter_grids = {
    'Preprocessed Data': param_grid_1,
    'Normalized Data': param_grid_2,
    'SelectKBest': param_grid_3,
    'RFECV': param_grid_4,
    'PCA': param_grid_5,
    'SMOTE + StandardScaler': param_grid_smote,
    'SMOTE + GridSearchCV': param_grid_smote,
    'RandomUnderSampler': param_grid_sampling,
    'TomekLinks': param_grid_sampling,
    'NearMiss': param_grid_sampling,
    'SMOTE + Tomek': param_grid_sampling,
}

# Initialize configurations list
configurations = []

# --- Configuration 1: Preprocessed Data ---
configurations.append(('Preprocessed Data', 'array', X_train_preprocessed, X_val_preprocessed))
print("✓ Configuration 1: Preprocessed Data")

# --- Configuration 2: Normalized Data (MinMax on preprocessed) ---
scaler_minmax = MinMaxScaler()
X_train_normalized = pd.DataFrame(
    scaler_minmax.fit_transform(X_train_preprocessed), 
    columns=X_train_preprocessed.columns, 
    index=X_train_preprocessed.index
)
X_val_normalized = pd.DataFrame(
    scaler_minmax.transform(X_val_preprocessed), 
    columns=X_val_preprocessed.columns, 
    index=X_val_preprocessed.index
)
configurations.append(('Normalized Data', 'array', X_train_normalized, X_val_normalized))
print("✓ Configuration 2: Normalized Data (MinMax)")

# --- Configuration 3: SelectKBest ---
print("\n=== SelectKBest Feature Selection ===")
scores = []
max_features = min(X_train_normalized.shape[1], 20)

for k in range(1, max_features + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_tr_k = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(
        SVC(kernel='linear', random_state=RANDOM_STATE), 
        X_tr_k, y_train, cv=5, scoring='accuracy', n_jobs=-1
    ).mean()
    scores.append(score)

optimal_k = int(np.argmax(scores) + 1)
print(f"Optimal number of features: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = pd.DataFrame(
    kbest.fit_transform(X_train_normalized, y_train), 
    columns=X_train_normalized.columns[kbest.get_support()]
)
X_val_kbest = pd.DataFrame(
    kbest.transform(X_val_normalized), 
    columns=X_train_kbest.columns
)
configurations.append(('SelectKBest', 'array', X_train_kbest, X_val_kbest))
print("✓ Configuration 3: SelectKBest")

# --- Configuration 4: RFECV ---
print("\n=== RFECV Feature Selection ===")
svm_estimator = SVC(kernel='linear', random_state=RANDOM_STATE)
rfecv = RFECV(
    estimator=svm_estimator, 
    step=1, 
    cv=StratifiedKFold(5), 
    scoring='accuracy', 
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=svm_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = pd.DataFrame(
    rfe.fit_transform(X_train_kbest, y_train), 
    columns=X_train_kbest.columns[rfe.get_support()]
)
X_val_rfe = pd.DataFrame(
    rfe.transform(X_val_kbest), 
    columns=X_train_rfe.columns
)
configurations.append(('RFECV', 'array', X_train_rfe, X_val_rfe))
print("✓ Configuration 4: RFECV")

# --- Configuration 5: PCA ---
print("\n=== PCA Dimensionality Reduction ===")
pca_full = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
desired_variance = 0.99
n_components = int(np.argmax(cumulative_variance >= desired_variance) + 1)
n_components = max(2, n_components)
print(f'Number of components for {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components, random_state=RANDOM_STATE)
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_rfe), index=X_train_rfe.index)
X_val_pca = pd.DataFrame(pca.transform(X_val_rfe), index=X_val_rfe.index)
configurations.append(('PCA', 'array', X_train_pca, X_val_pca))
print("✓ Configuration 5: PCA")

# --- Configuration 6: SMOTE + StandardScaler (Pipeline) ---
pipeline_smote_scaler = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + StandardScaler', 'pipeline', pipeline_smote_scaler, None))
print("✓ Configuration 6: SMOTE + StandardScaler (Pipeline)")

# --- Configuration 7: SMOTE + GridSearchCV (Pipeline) ---
pipeline_smote_grid = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + GridSearchCV', 'pipeline', pipeline_smote_grid, None))
print("✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)")

# =============================================================================
# UNDERSAMPLING CONFIGURATIONS
# =============================================================================
print("\n=== Adding Undersampling Configurations ===")

# --- Configuration 8: RandomUnderSampler ---
pipeline_rus = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', RandomUnderSampler(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('RandomUnderSampler', 'pipeline', pipeline_rus, None))
print("✓ Configuration 8: RandomUnderSampler (Undersampling)")

# --- Configuration 9: TomekLinks ---
pipeline_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', TomekLinks()),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('TomekLinks', 'pipeline', pipeline_tomek, None))
print("✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)")

# --- Configuration 10: NearMiss ---
pipeline_nearmiss = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', NearMiss(version=1)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('NearMiss', 'pipeline', pipeline_nearmiss, None))
print("✓ Configuration 10: NearMiss (Undersampling - selective)")

# --- Configuration 11: SMOTE + Tomek (Combined) ---
pipeline_smote_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('sampler', SMOTETomek(random_state=RANDOM_STATE)),
    ('model', SVC(probability=True, random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + Tomek', 'pipeline', pipeline_smote_tomek, None))
print("✓ Configuration 11: SMOTE + Tomek (Combined Over + Under)")

print(f"\nTotal configurations: {len(configurations)}")

# Safe ROC AUC helper
def safe_roc_auc(y_true, y_proba):
    try:
        if isinstance(y_proba, np.ndarray) and y_proba.shape[1] == 2:
            return metrics.roc_auc_score(y_true, y_proba[:, 1])
        else:
            return metrics.roc_auc_score(
                pd.get_dummies(y_true), y_proba, 
                multi_class='ovr', average='macro'
            )
    except Exception:
        return np.nan

# =============================================================================
# Run SVM with Configuration-Specific GridSearchCV
# =============================================================================
print("\n" + "="*80)
print("RUNNING SVM WITH CONFIGURATION-SPECIFIC HYPERPARAMETER TUNING")
print("="*80)

for name, kind, X_tr_cfg, X_val_cfg in configurations:
    print(f"\n{'='*80}")
    print(f"Configuration: {name}")
    print(f"{'='*80}")
    
    # Get the specific parameter grid for this configuration
    param_grid = hyperparameter_grids[name]
    
    print(f"Hyperparameter grid for '{name}':")
    for key, values in param_grid.items():
        print(f"  {key}: {values}")
    print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")
    
    try:
        if kind == 'pipeline':
            pipeline = X_tr_cfg
            grid_search = GridSearchCV(
                pipeline, param_grid, 
                cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_train)
            y_val_pred = best_model.predict(X_val)
            y_train_proba = best_model.predict_proba(X_train)
            y_val_proba = best_model.predict_proba(X_val)
        else:
            grid_search = GridSearchCV(
                SVC(probability=True, random_state=RANDOM_STATE), 
                param_grid, cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_tr_cfg, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_tr_cfg)
            y_val_pred = best_model.predict(X_val_cfg)
            y_train_proba = best_model.predict_proba(X_tr_cfg)
            y_val_proba = best_model.predict_proba(X_val_cfg)

        # Calculate train-test gap for overfitting detection
        train_acc = metrics.accuracy_score(y_train, y_train_pred)
        test_acc = metrics.accuracy_score(y_val, y_val_pred)
        train_test_gap = train_acc - test_acc

        # Build metrics dictionary
        metrics_dict = {
            "Dataset": ["Training", "Test"],
            "Accuracy": [train_acc, test_acc],
            "F1 Score": [
                metrics.f1_score(y_train, y_train_pred, average='macro'),
                metrics.f1_score(y_val, y_val_pred, average='macro'),
            ],
            "Recall": [
                metrics.recall_score(y_train, y_train_pred, average='macro'),
                metrics.recall_score(y_val, y_val_pred, average='macro'),
            ],
            "Precision": [
                metrics.precision_score(y_train, y_train_pred, average='macro', zero_division=0),
                metrics.precision_score(y_val, y_val_pred, average='macro', zero_division=0),
            ],
            "AUC-ROC": [
                safe_roc_auc(y_train, y_train_proba),
                safe_roc_auc(y_val, y_val_proba),
            ]
        }

        df_metrics = pd.DataFrame(metrics_dict)
        pd.options.display.float_format = '{:.6f}'.format
        print("\n📊 Support Vector Machine Model Performance Metrics")
        print(df_metrics.to_string(index=False))

        # Overfitting warning
        if train_test_gap > 0.10:
            print(f"\n⚠️  WARNING: Overfitting detected! Train-Test gap: {train_test_gap:.4f}")
        elif train_test_gap < 0.05:
            print(f"\n✓ Good generalization. Train-Test gap: {train_test_gap:.4f}")
        else:
            print(f"\n→ Acceptable gap: {train_test_gap:.4f}")

        # Store test metrics
        test_metrics = df_metrics[df_metrics['Dataset'] == 'Test'].iloc[0]
        try:
            store_results(
                'SVM',
                name,
                float(test_metrics['Accuracy']),
                float(test_metrics['F1 Score']),
                float(test_metrics['Recall']),
                float(test_metrics['Precision']),
                float(test_metrics['AUC-ROC'])
            )
        except:
            ML_Model.append('SVM')
            ML_Config.append(name)
            accuracy.append(round(float(test_metrics['Accuracy']), 6))
            f1.append(round(float(test_metrics['F1 Score']), 6))
            recall.append(round(float(test_metrics['Recall']), 6))
            precision.append(round(float(test_metrics['Precision']), 6))
            roc_auc.append(round(float(test_metrics['AUC-ROC']), 6))

        print("\n🎯 Best hyperparameters found:")
        best_params = grid_search.best_params_
        for param, value in best_params.items():
            print(f"  {param}: {value}")
        print(f"\nBest CV score: {grid_search.best_score_:.6f}")
        
    except Exception as e:
        print(f"❌ Error in configuration '{name}': {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*80)
print("✅ SVM evaluation complete for all configurations.")
print("="*80)

# Display final results
try:
    display_and_save_results('svm_all_configs')
except:
    result = pd.DataFrame({
        'ML Model': ML_Model,
        'Configuration': ML_Config,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Recall': recall,
        'Precision': precision,
        'ROC_AUC': roc_auc,
    })
    print("\n📈 Final Results:")
    print(result.to_string(index=False))
    
    # Sort by F1 Score
    sorted_result = result.sort_values(by=['F1 Score', 'Accuracy'], ascending=False).reset_index(drop=True)
    print("\n🏆 Sorted Results (by F1 Score):")
    print(sorted_result.to_string(index=False))

    # Group by sampling technique
    print("\n📊 Performance by Sampling Technique:")
    print("-" * 80)
    sampling_groups = {
        'No Sampling': ['Preprocessed Data', 'Normalized Data', 'SelectKBest', 'RFECV', 'PCA'],
        'Oversampling': ['SMOTE + StandardScaler', 'SMOTE + GridSearchCV'],
        'Undersampling': ['RandomUnderSampler', 'TomekLinks', 'NearMiss'],
        'Combined': ['SMOTE + Tomek']
    }

    for group_name, configs in sampling_groups.items():
        group_data = result[result['Configuration'].isin(configs)]
        if not group_data.empty:
            print(f"\n{group_name}:")
            print(f"  Avg Accuracy: {group_data['Accuracy'].mean():.6f}")
            print(f"  Avg F1 Score: {group_data['F1 Score'].mean():.6f}")
            print(f"  Avg ROC-AUC: {group_data['ROC_AUC'].mean():.6f}")
            print(f"  Best Config: {group_data.loc[group_data['F1 Score'].idxmax(), 'Configuration']}")



Numeric features: ['Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'Delay', 'Visit', 'MR Delay']
Categorical features: ['M/F']

Preprocessed data shape: (647, 13)
All features are now numeric: True
Results cleared!

=== START: SVM Configuration Sweep with Custom Hyperparameters ===

✓ Configuration 1: Preprocessed Data
✓ Configuration 2: Normalized Data (MinMax)

=== SelectKBest Feature Selection ===
Optimal number of features: 10
✓ Configuration 3: SelectKBest

=== RFECV Feature Selection ===
Optimal number of features by RFECV: 3
✓ Configuration 4: RFECV

=== PCA Dimensionality Reduction ===
Number of components for 99.0% variance: 3
✓ Configuration 5: PCA
✓ Configuration 6: SMOTE + StandardScaler (Pipeline)
✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)

=== Adding Undersampling Configurations ===
✓ Configuration 8: RandomUnderSampler (Undersampling)
✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)
✓ Configuration 10: NearMiss (Undersampling - se

---

# Random Forest

### Random Forest with PCA 90

In [10]:
# =============================================================================
# Random Forest with Configuration-Specific Hyperparameter Grids
# =============================================================================
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE, RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn import metrics
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.combine import SMOTETomek
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Define preprocessor for categorical and numeric features
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# Create preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

# Apply preprocessing to get fully numeric data first
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

# Get feature names after preprocessing
try:
    feature_names = (numeric_features + 
                    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))
except:
    feature_names = [f'feature_{i}' for i in range(X_train_preprocessed.shape[1])]

X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=feature_names, index=X_train.index)
X_val_preprocessed = pd.DataFrame(X_val_preprocessed, columns=feature_names, index=X_val.index)

print(f"\nPreprocessed data shape: {X_train_preprocessed.shape}")
print(f"All features are now numeric: {X_train_preprocessed.select_dtypes(include=np.number).shape[1] == X_train_preprocessed.shape[1]}")

# Clear previous results
try:
    clear_results()
except:
    ML_Model = []
    ML_Config = []
    accuracy = []
    f1 = []
    recall = []
    precision = []
    roc_auc = []
    print("Initialized result storage lists.")

print("\n=== START: Random Forest Configuration Sweep with Custom Hyperparameters ===\n")

# =============================================================================
# Configuration-Specific Hyperparameter Grids for Random Forest
# =============================================================================

# Grid 1: Preprocessed Data - Balanced exploration
param_grid_1 = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True]
}

# Grid 2: Normalized Data - Focus on deeper trees
param_grid_2 = {
    'n_estimators': [100, 200, 300],
    'max_depth': [15, 25, 35, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

# Grid 3: SelectKBest - More conservative (prevent overfitting)
param_grid_3 = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [2, 4, 6],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini'],
    'bootstrap': [True]
}

# Grid 4: RFECV - Focus on feature importance
param_grid_4 = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True]
}

# Grid 5: PCA - Simpler models for reduced dimensions
param_grid_5 = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [10, 15, 20, 25, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

# Grid 6 & 7: SMOTE pipelines - Handle imbalanced data (Oversampling)
param_grid_smote = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['sqrt', 'log2'],
    'model__criterion': ['gini', 'entropy'],
    'model__bootstrap': [True]
}

# Grid 8-11: Undersampling and Combined pipelines
param_grid_sampling = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['sqrt', 'log2'],
    'model__criterion': ['gini', 'entropy'],
    'model__bootstrap': [True]
}

# Map grids to configurations
hyperparameter_grids = {
    'Preprocessed Data': param_grid_1,
    'Normalized Data': param_grid_2,
    'SelectKBest': param_grid_3,
    'RFECV': param_grid_4,
    'PCA': param_grid_5,
    'SMOTE + StandardScaler': param_grid_smote,
    'SMOTE + GridSearchCV': param_grid_smote,
    'RandomUnderSampler': param_grid_sampling,
    'TomekLinks': param_grid_sampling,
    'NearMiss': param_grid_sampling,
    'SMOTE + Tomek': param_grid_sampling,
}

# Initialize configurations list
configurations = []

# --- Configuration 1: Preprocessed Data ---
configurations.append(('Preprocessed Data', 'array', X_train_preprocessed, X_val_preprocessed))
print("✓ Configuration 1: Preprocessed Data")

# --- Configuration 2: Normalized Data (MinMax on preprocessed) ---
scaler_minmax = MinMaxScaler()
X_train_normalized = pd.DataFrame(
    scaler_minmax.fit_transform(X_train_preprocessed), 
    columns=X_train_preprocessed.columns, 
    index=X_train_preprocessed.index
)
X_val_normalized = pd.DataFrame(
    scaler_minmax.transform(X_val_preprocessed), 
    columns=X_val_preprocessed.columns, 
    index=X_val_preprocessed.index
)
configurations.append(('Normalized Data', 'array', X_train_normalized, X_val_normalized))
print("✓ Configuration 2: Normalized Data (MinMax)")

# --- Configuration 3: SelectKBest ---
print("\n=== SelectKBest Feature Selection ===")
scores = []
max_features = min(X_train_normalized.shape[1], 20)

for k in range(1, max_features + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_tr_k = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(
        RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE), 
        X_tr_k, y_train, cv=5, scoring='accuracy', n_jobs=-1
    ).mean()
    scores.append(score)

optimal_k = int(np.argmax(scores) + 1)
print(f"Optimal number of features: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = pd.DataFrame(
    kbest.fit_transform(X_train_normalized, y_train), 
    columns=X_train_normalized.columns[kbest.get_support()]
)
X_val_kbest = pd.DataFrame(
    kbest.transform(X_val_normalized), 
    columns=X_train_kbest.columns
)
configurations.append(('SelectKBest', 'array', X_train_kbest, X_val_kbest))
print("✓ Configuration 3: SelectKBest")

# --- Configuration 4: RFECV ---
print("\n=== RFECV Feature Selection ===")
rf_estimator = RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE)
rfecv = RFECV(
    estimator=rf_estimator, 
    step=1, 
    cv=StratifiedKFold(5), 
    scoring='accuracy', 
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=rf_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = pd.DataFrame(
    rfe.fit_transform(X_train_kbest, y_train), 
    columns=X_train_kbest.columns[rfe.get_support()]
)
X_val_rfe = pd.DataFrame(
    rfe.transform(X_val_kbest), 
    columns=X_train_rfe.columns
)
configurations.append(('RFECV', 'array', X_train_rfe, X_val_rfe))
print("✓ Configuration 4: RFECV")

# --- Configuration 5: PCA ---
print("\n=== PCA Dimensionality Reduction ===")
pca_full = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
desired_variance = 0.90
n_components = int(np.argmax(cumulative_variance >= desired_variance) + 1)
n_components = max(2, n_components)
print(f'Number of components for {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components, random_state=RANDOM_STATE)
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_rfe), index=X_train_rfe.index)
X_val_pca = pd.DataFrame(pca.transform(X_val_rfe), index=X_val_rfe.index)
configurations.append(('PCA', 'array', X_train_pca, X_val_pca))
print("✓ Configuration 5: PCA")

# --- Configuration 6: SMOTE + StandardScaler (Pipeline) ---
pipeline_smote_scaler = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + StandardScaler', 'pipeline', pipeline_smote_scaler, None))
print("✓ Configuration 6: SMOTE + StandardScaler (Pipeline)")

# --- Configuration 7: SMOTE + GridSearchCV (Pipeline) ---
pipeline_smote_grid = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + GridSearchCV', 'pipeline', pipeline_smote_grid, None))
print("✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)")

# =============================================================================
# UNDERSAMPLING CONFIGURATIONS
# =============================================================================
print("\n=== Adding Undersampling Configurations ===")

# --- Configuration 8: RandomUnderSampler ---
pipeline_rus = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', RandomUnderSampler(random_state=RANDOM_STATE)),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE))
])
configurations.append(('RandomUnderSampler', 'pipeline', pipeline_rus, None))
print("✓ Configuration 8: RandomUnderSampler (Undersampling)")

# --- Configuration 9: TomekLinks ---
pipeline_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', TomekLinks()),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE))
])
configurations.append(('TomekLinks', 'pipeline', pipeline_tomek, None))
print("✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)")

# --- Configuration 10: NearMiss ---
pipeline_nearmiss = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('undersampler', NearMiss(version=1)),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE))
])
configurations.append(('NearMiss', 'pipeline', pipeline_nearmiss, None))
print("✓ Configuration 10: NearMiss (Undersampling - selective)")

# --- Configuration 11: SMOTE + Tomek (Combined) ---
pipeline_smote_tomek = ImbPipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])),
    ('sampler', SMOTETomek(random_state=RANDOM_STATE)),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE))
])
configurations.append(('SMOTE + Tomek', 'pipeline', pipeline_smote_tomek, None))
print("✓ Configuration 11: SMOTE + Tomek (Combined Over + Under)")

print(f"\nTotal configurations: {len(configurations)}")

# Safe ROC AUC helper
def safe_roc_auc(y_true, y_proba):
    try:
        if isinstance(y_proba, np.ndarray) and y_proba.shape[1] == 2:
            return metrics.roc_auc_score(y_true, y_proba[:, 1])
        else:
            return metrics.roc_auc_score(
                pd.get_dummies(y_true), y_proba, 
                multi_class='ovr', average='macro'
            )
    except Exception:
        return np.nan

# =============================================================================
# Run Random Forest with Configuration-Specific GridSearchCV
# =============================================================================
print("\n" + "="*80)
print("RUNNING RANDOM FOREST WITH CONFIGURATION-SPECIFIC HYPERPARAMETER TUNING")
print("="*80)

for name, kind, X_tr_cfg, X_val_cfg in configurations:
    print(f"\n{'='*80}")
    print(f"Configuration: {name}")
    print(f"{'='*80}")
    
    # Get the specific parameter grid for this configuration
    param_grid = hyperparameter_grids[name]
    
    print(f"Hyperparameter grid for '{name}':")
    for key, values in param_grid.items():
        print(f"  {key}: {values}")
    print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")
    
    try:
        if kind == 'pipeline':
            pipeline = X_tr_cfg
            grid_search = GridSearchCV(
                pipeline, param_grid, 
                cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_train)
            y_val_pred = best_model.predict(X_val)
            y_train_proba = best_model.predict_proba(X_train)
            y_val_proba = best_model.predict_proba(X_val)
        else:
            grid_search = GridSearchCV(
                RandomForestClassifier(random_state=RANDOM_STATE), 
                param_grid, cv=5, n_jobs=-1, verbose=1, scoring='f1_macro'
            )
            grid_search.fit(X_tr_cfg, y_train)
            best_model = grid_search.best_estimator_
            y_train_pred = best_model.predict(X_tr_cfg)
            y_val_pred = best_model.predict(X_val_cfg)
            y_train_proba = best_model.predict_proba(X_tr_cfg)
            y_val_proba = best_model.predict_proba(X_val_cfg)

        # Calculate train-test gap for overfitting detection
        train_acc = metrics.accuracy_score(y_train, y_train_pred)
        test_acc = metrics.accuracy_score(y_val, y_val_pred)
        train_test_gap = train_acc - test_acc

        # Build metrics dictionary
        metrics_dict = {
            "Dataset": ["Training", "Test"],
            "Accuracy": [train_acc, test_acc],
            "F1 Score": [
                metrics.f1_score(y_train, y_train_pred, average='macro'),
                metrics.f1_score(y_val, y_val_pred, average='macro'),
            ],
            "Recall": [
                metrics.recall_score(y_train, y_train_pred, average='macro'),
                metrics.recall_score(y_val, y_val_pred, average='macro'),
            ],
            "Precision": [
                metrics.precision_score(y_train, y_train_pred, average='macro', zero_division=0),
                metrics.precision_score(y_val, y_val_pred, average='macro', zero_division=0),
            ],
            "AUC-ROC": [
                safe_roc_auc(y_train, y_train_proba),
                safe_roc_auc(y_val, y_val_proba),
            ]
        }

        df_metrics = pd.DataFrame(metrics_dict)
        pd.options.display.float_format = '{:.6f}'.format
        print("\n📊 Random Forest Model Performance Metrics")
        print(df_metrics.to_string(index=False))

        # Overfitting warning
        if train_test_gap > 0.10:
            print(f"\n⚠️  WARNING: Overfitting detected! Train-Test gap: {train_test_gap:.4f}")
        elif train_test_gap < 0.05:
            print(f"\n✓ Good generalization. Train-Test gap: {train_test_gap:.4f}")
        else:
            print(f"\n→ Acceptable gap: {train_test_gap:.4f}")

        # Store test metrics
        test_metrics = df_metrics[df_metrics['Dataset'] == 'Test'].iloc[0]
        try:
            store_results(
                'Random Forest',
                name,
                float(test_metrics['Accuracy']),
                float(test_metrics['F1 Score']),
                float(test_metrics['Recall']),
                float(test_metrics['Precision']),
                float(test_metrics['AUC-ROC'])
            )
        except:
            ML_Model.append('Random Forest')
            ML_Config.append(name)
            accuracy.append(round(float(test_metrics['Accuracy']), 6))
            f1.append(round(float(test_metrics['F1 Score']), 6))
            recall.append(round(float(test_metrics['Recall']), 6))
            precision.append(round(float(test_metrics['Precision']), 6))
            roc_auc.append(round(float(test_metrics['AUC-ROC']), 6))

        print("\n🎯 Best hyperparameters found:")
        best_params = grid_search.best_params_
        for param, value in best_params.items():
            print(f"  {param}: {value}")
        print(f"\nBest CV score: {grid_search.best_score_:.6f}")
        
        # Feature importance for non-pipeline configurations
        if kind == 'array' and hasattr(best_model, 'feature_importances_'):
            print("\n🌲 Top 10 Most Important Features:")
            importances = best_model.feature_importances_
            indices = np.argsort(importances)[::-1][:10]
            feature_cols = X_tr_cfg.columns if hasattr(X_tr_cfg, 'columns') else [f'Feature {i}' for i in range(len(importances))]
            for i, idx in enumerate(indices, 1):
                print(f"  {i}. {feature_cols[idx]}: {importances[idx]:.4f}")
        
    except Exception as e:
        print(f"❌ Error in configuration '{name}': {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*80)
print("✅ Random Forest evaluation complete for all configurations.")
print("="*80)

# Display final results
try:
    display_and_save_results('random_forest_all_configs')
except:
    result = pd.DataFrame({
        'ML Model': ML_Model,
        'Configuration': ML_Config,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Recall': recall,
        'Precision': precision,
        'ROC_AUC': roc_auc,
    })
    print("\n📈 Final Results:")
    print(result.to_string(index=False))
    
    # Sort by F1 Score
    sorted_result = result.sort_values(by=['F1 Score', 'Accuracy'], ascending=False).reset_index(drop=True)
    print("\n🏆 Sorted Results (by F1 Score):")
    print(sorted_result.to_string(index=False))

    # Group by sampling technique
    print("\n📊 Performance by Sampling Technique:")
    print("-" * 80)
    sampling_groups = {
        'No Sampling': ['Preprocessed Data', 'Normalized Data', 'SelectKBest', 'RFECV', 'PCA'],
        'Oversampling': ['SMOTE + StandardScaler', 'SMOTE + GridSearchCV'],
        'Undersampling': ['RandomUnderSampler', 'TomekLinks', 'NearMiss'],
        'Combined': ['SMOTE + Tomek']
    }

    for group_name, configs in sampling_groups.items():
        group_data = result[result['Configuration'].isin(configs)]
        if not group_data.empty:
            print(f"\n{group_name}:")
            print(f"  Avg Accuracy: {group_data['Accuracy'].mean():.6f}")
            print(f"  Avg F1 Score: {group_data['F1 Score'].mean():.6f}")
            print(f"  Avg ROC-AUC: {group_data['ROC_AUC'].mean():.6f}")
            print(f"  Best Config: {group_data.loc[group_data['F1 Score'].idxmax(), 'Configuration']}")
    
    # Compare with baseline
    if len(result) > 0:
        best_idx = result['F1 Score'].idxmax()
        print(f"\n🏅 Best Overall Configuration:")
        print(f"  Configuration: {result.loc[best_idx, 'Configuration']}")
        print(f"  F1 Score: {result.loc[best_idx, 'F1 Score']:.6f}")
        print(f"  Accuracy: {result.loc[best_idx, 'Accuracy']:.6f}")
        print(f"  ROC-AUC: {result.loc[best_idx, 'ROC_AUC']:.6f}")


Numeric features: ['Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'Delay', 'Visit', 'MR Delay']
Categorical features: ['M/F']

Preprocessed data shape: (647, 13)
All features are now numeric: True
Results cleared!

=== START: Random Forest Configuration Sweep with Custom Hyperparameters ===

✓ Configuration 1: Preprocessed Data
✓ Configuration 2: Normalized Data (MinMax)

=== SelectKBest Feature Selection ===
Optimal number of features: 12
✓ Configuration 3: SelectKBest

=== RFECV Feature Selection ===
Optimal number of features by RFECV: 2
✓ Configuration 4: RFECV

=== PCA Dimensionality Reduction ===
Number of components for 90.0% variance: 2
✓ Configuration 5: PCA
✓ Configuration 6: SMOTE + StandardScaler (Pipeline)
✓ Configuration 7: SMOTE + GridSearchCV (Pipeline)

=== Adding Undersampling Configurations ===
✓ Configuration 8: RandomUnderSampler (Undersampling)
✓ Configuration 9: TomekLinks (Undersampling - removes noisy samples)
✓ Configuration 10: NearMiss (Undersam

### Random Forest with PCA 95

### Random Forest with PCA 99

---

# Gradient Boosting

### Gradient Boosting with PCA 90

In [14]:
# Written by Ovi, 2025-07-07, Gradient Boosting classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(GradientBoostingClassifier(), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with Gradient Boosting ===")
gbc_estimator = GradientBoostingClassifier()

rfecv = RFECV(estimator=gbc_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=gbc_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.90
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: Gradient Boosting + GridSearchCV
print("\n=== Gradient Boosting Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200,500],
    'max_depth': [3, 5,7,9,15, 21],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'subsample': [0.8],
    'max_features': ['sqrt']
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Gradient Boosting with {name} configuration...")
    gbc = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1, verbose=2)
    gbc.fit(X_train_cfg, y_train_cfg)

    y_train_gbc = gbc.predict(X_train_cfg)
    y_test_gbc = gbc.predict(X_test_cfg)
    y_train_gbc_proba = gbc.predict_proba(X_train_cfg)
    y_test_gbc_proba = gbc.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_gbc),
            metrics.accuracy_score(y_test, y_test_gbc),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.f1_score(y_test, y_test_gbc, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.recall_score(y_test, y_test_gbc, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.precision_score(y_test, y_test_gbc, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_gbc_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_gbc_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nGradient Boosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_gbc_proba, multi_class='ovr', average='macro')
    storeResults(
        'Gradient Boosting 90',
        name,
        metrics.accuracy_score(y_test, y_test_gbc),
        metrics.f1_score(y_test, y_test_gbc, average='macro'),
        metrics.recall_score(y_test, y_test_gbc, average='macro'),
        metrics.precision_score(y_test, y_test_gbc, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(gbc.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 14

=== RFECV Feature Selection with Gradient Boosting ===
Optimal number of features selected by RFECV: 14

=== PCA Dimensionality Reduction ===
Number of components that explain 90.0% variance: 12

=== Gradient Boosting Model Performance with Hyperparameter Tuning ===

Running Gradient Boosting with Original Data configuration...
Fitting 10 folds for each of 144 candidates, totalling 1440 fits

Gradient Boosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.966825  0.848374 0.765152   0.988275 0.990586
Best hyperparameters found by GridSearchCV:
{'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500, 'subsample': 0.8}

Running Gradient Boosting with Normalized Data configuration...
Fitting 10 folds for each of 

### Gradient Boosting with PCA 95

In [15]:
# Written by Ovi, 2025-07-07, Gradient Boosting classification with preprocessing and result logging


# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(GradientBoostingClassifier(), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with Gradient Boosting ===")
gbc_estimator = GradientBoostingClassifier()

rfecv = RFECV(estimator=gbc_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=gbc_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.95
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: Gradient Boosting + GridSearchCV
print("\n=== Gradient Boosting Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200,500],
    'max_depth': [3, 5,7,9,15, 21],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'subsample': [0.8],
    'max_features': ['sqrt']
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Gradient Boosting with {name} configuration...")
    gbc = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1, verbose=2)
    gbc.fit(X_train_cfg, y_train_cfg)

    y_train_gbc = gbc.predict(X_train_cfg)
    y_test_gbc = gbc.predict(X_test_cfg)
    y_train_gbc_proba = gbc.predict_proba(X_train_cfg)
    y_test_gbc_proba = gbc.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_gbc),
            metrics.accuracy_score(y_test, y_test_gbc),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.f1_score(y_test, y_test_gbc, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.recall_score(y_test, y_test_gbc, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.precision_score(y_test, y_test_gbc, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_gbc_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_gbc_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nGradient Boosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_gbc_proba, multi_class='ovr', average='macro')
    storeResults(
        'Gradient Boosting 95',
        name,
        metrics.accuracy_score(y_test, y_test_gbc),
        metrics.f1_score(y_test, y_test_gbc, average='macro'),
        metrics.recall_score(y_test, y_test_gbc, average='macro'),
        metrics.precision_score(y_test, y_test_gbc, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(gbc.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 16

=== RFECV Feature Selection with Gradient Boosting ===
Optimal number of features selected by RFECV: 16

=== PCA Dimensionality Reduction ===
Number of components that explain 95.0% variance: 15

=== Gradient Boosting Model Performance with Hyperparameter Tuning ===

Running Gradient Boosting with Original Data configuration...
Fitting 10 folds for each of 144 candidates, totalling 1440 fits

Gradient Boosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000   1.00000 1.000000    1.00000 1.000000
    Test  0.947867   0.74708 0.661143    0.91675 0.986373
Best hyperparameters found by GridSearchCV:
{'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200, 'subsample': 0.8}

Running Gradient Boosting with Normalized Data configuration...
Fitting 10 folds for each of 

### Gradient Boosting with PCA 99

In [16]:
# Written by Ovi, 2025-07-07, Gradient Boosting classification with preprocessing and result logging


# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(GradientBoostingClassifier(), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with Gradient Boosting ===")
gbc_estimator = GradientBoostingClassifier()

rfecv = RFECV(estimator=gbc_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=gbc_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.99
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: Gradient Boosting + GridSearchCV
print("\n=== Gradient Boosting Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200,500],
    'max_depth': [3, 5,7,9,15, 21],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'subsample': [0.8],
    'max_features': ['sqrt']
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Gradient Boosting with {name} configuration...")
    gbc = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1, verbose=2)
    gbc.fit(X_train_cfg, y_train_cfg)

    y_train_gbc = gbc.predict(X_train_cfg)
    y_test_gbc = gbc.predict(X_test_cfg)
    y_train_gbc_proba = gbc.predict_proba(X_train_cfg)
    y_test_gbc_proba = gbc.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_gbc),
            metrics.accuracy_score(y_test, y_test_gbc),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.f1_score(y_test, y_test_gbc, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.recall_score(y_test, y_test_gbc, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_gbc, average='macro'),
            metrics.precision_score(y_test, y_test_gbc, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_gbc_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_gbc_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nGradient Boosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_gbc_proba, multi_class='ovr', average='macro')
    storeResults(
        'Gradient Boosting 99',
        name,
        metrics.accuracy_score(y_test, y_test_gbc),
        metrics.f1_score(y_test, y_test_gbc, average='macro'),
        metrics.recall_score(y_test, y_test_gbc, average='macro'),
        metrics.precision_score(y_test, y_test_gbc, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(gbc.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 14

=== RFECV Feature Selection with Gradient Boosting ===
Optimal number of features selected by RFECV: 14

=== PCA Dimensionality Reduction ===
Number of components that explain 99.0% variance: 14

=== Gradient Boosting Model Performance with Hyperparameter Tuning ===

Running Gradient Boosting with Original Data configuration...
Fitting 10 folds for each of 144 candidates, totalling 1440 fits

Gradient Boosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.962085  0.820578 0.734848   0.986667 0.990144
Best hyperparameters found by GridSearchCV:
{'learning_rate': 0.05, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500, 'subsample': 0.8}

Running Gradient Boosting with Normalized Data configuration...
Fitting 10 folds for each of

---

# Adaboost

### Adaboost with PCA 90

In [17]:
# Written by Ovi, 2025-07-07, AdaBoost classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(AdaBoostClassifier(), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with AdaBoost ===")
ab_estimator = AdaBoostClassifier()

rfecv = RFECV(estimator=ab_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=ab_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.90
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: AdaBoost + GridSearchCV
print("\n=== AdaBoost Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 1],
    'algorithm': ['SAMME', 'SAMME.R'],
    'estimator': [DecisionTreeClassifier(max_depth=d) for d in [1, 3, 5]]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning AdaBoost with {name} configuration...")
    ab = GridSearchCV(
        AdaBoostClassifier(),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    ab.fit(X_train_cfg, y_train_cfg)

    y_train_ab = ab.predict(X_train_cfg)
    y_test_ab = ab.predict(X_test_cfg)
    y_train_ab_proba = ab.predict_proba(X_train_cfg)
    y_test_ab_proba = ab.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_ab),
            metrics.accuracy_score(y_test, y_test_ab),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.f1_score(y_test, y_test_ab, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.recall_score(y_test, y_test_ab, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.precision_score(y_test, y_test_ab, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_ab_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_ab_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nAdaBoost Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_ab_proba, multi_class='ovr', average='macro')
    storeResults(
        'AdaBoost 90',
        name,
        metrics.accuracy_score(y_test, y_test_ab),
        metrics.f1_score(y_test, y_test_ab, average='macro'),
        metrics.recall_score(y_test, y_test_ab, average='macro'),
        metrics.precision_score(y_test, y_test_ab, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(ab.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 25

=== RFECV Feature Selection with AdaBoost ===
Optimal number of features selected by RFECV: 11

=== PCA Dimensionality Reduction ===
Number of components that explain 90.0% variance: 10

=== AdaBoost Model Performance with Hyperparameter Tuning ===

Running AdaBoost with Original Data configuration...
Fitting 10 folds for each of 72 candidates, totalling 720 fits

AdaBoost Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.966825  0.860668 0.805082   0.942229 0.992096
Best hyperparameters found by GridSearchCV:
{'algorithm': 'SAMME', 'estimator': DecisionTreeClassifier(max_depth=3), 'learning_rate': 1, 'n_estimators': 200}

Running AdaBoost with Normalized Data configuration...
Fitting 10 folds for each of 72 candidates, totalling 720 fits

AdaBoost Model Performance Metrics
 Dataset  Ac

### Adaboost with PCA 95

In [18]:
# Written by Ovi, 2025-07-07, AdaBoost classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(AdaBoostClassifier(), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with AdaBoost ===")
ab_estimator = AdaBoostClassifier()

rfecv = RFECV(estimator=ab_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=ab_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.95
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: AdaBoost + GridSearchCV
print("\n=== AdaBoost Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 1],
    'algorithm': ['SAMME', 'SAMME.R'],
    'estimator': [DecisionTreeClassifier(max_depth=d) for d in [1, 3, 5]]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning AdaBoost with {name} configuration...")
    ab = GridSearchCV(
        AdaBoostClassifier(),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    ab.fit(X_train_cfg, y_train_cfg)

    y_train_ab = ab.predict(X_train_cfg)
    y_test_ab = ab.predict(X_test_cfg)
    y_train_ab_proba = ab.predict_proba(X_train_cfg)
    y_test_ab_proba = ab.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_ab),
            metrics.accuracy_score(y_test, y_test_ab),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.f1_score(y_test, y_test_ab, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.recall_score(y_test, y_test_ab, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.precision_score(y_test, y_test_ab, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_ab_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_ab_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nAdaBoost Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_ab_proba, multi_class='ovr', average='macro')
    storeResults(
        'AdaBoost 95',
        name,
        metrics.accuracy_score(y_test, y_test_ab),
        metrics.f1_score(y_test, y_test_ab, average='macro'),
        metrics.recall_score(y_test, y_test_ab, average='macro'),
        metrics.precision_score(y_test, y_test_ab, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(ab.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 25

=== RFECV Feature Selection with AdaBoost ===
Optimal number of features selected by RFECV: 11

=== PCA Dimensionality Reduction ===
Number of components that explain 95.0% variance: 10

=== AdaBoost Model Performance with Hyperparameter Tuning ===

Running AdaBoost with Original Data configuration...
Fitting 10 folds for each of 72 candidates, totalling 720 fits

AdaBoost Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.957346  0.790443 0.693182   0.985075 0.984545
Best hyperparameters found by GridSearchCV:
{'algorithm': 'SAMME', 'estimator': DecisionTreeClassifier(max_depth=3), 'learning_rate': 1, 'n_estimators': 100}

Running AdaBoost with Normalized Data configuration...
Fitting 10 folds for each of 72 candidates, totalling 720 fits

AdaBoost Model Performance Metrics
 Dataset  Ac

### Adaboost with PCA 99

In [19]:
# Written by Ovi, 2025-07-07, AdaBoost classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(AdaBoostClassifier(), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with AdaBoost ===")
ab_estimator = AdaBoostClassifier()

rfecv = RFECV(estimator=ab_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=ab_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.99
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: AdaBoost + GridSearchCV
print("\n=== AdaBoost Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 1],
    'algorithm': ['SAMME', 'SAMME.R'],
    'estimator': [DecisionTreeClassifier(max_depth=d) for d in [1, 3, 5]]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning AdaBoost with {name} configuration...")
    ab = GridSearchCV(
        AdaBoostClassifier(),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    ab.fit(X_train_cfg, y_train_cfg)

    y_train_ab = ab.predict(X_train_cfg)
    y_test_ab = ab.predict(X_test_cfg)
    y_train_ab_proba = ab.predict_proba(X_train_cfg)
    y_test_ab_proba = ab.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_ab),
            metrics.accuracy_score(y_test, y_test_ab),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.f1_score(y_test, y_test_ab, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.recall_score(y_test, y_test_ab, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_ab, average='macro'),
            metrics.precision_score(y_test, y_test_ab, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_ab_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_ab_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nAdaBoost Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_ab_proba, multi_class='ovr', average='macro')
    storeResults(
        'AdaBoost 99',
        name,
        metrics.accuracy_score(y_test, y_test_ab),
        metrics.f1_score(y_test, y_test_ab, average='macro'),
        metrics.recall_score(y_test, y_test_ab, average='macro'),
        metrics.precision_score(y_test, y_test_ab, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(ab.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 25

=== RFECV Feature Selection with AdaBoost ===
Optimal number of features selected by RFECV: 11

=== PCA Dimensionality Reduction ===
Number of components that explain 99.0% variance: 11

=== AdaBoost Model Performance with Hyperparameter Tuning ===

Running AdaBoost with Original Data configuration...
Fitting 10 folds for each of 72 candidates, totalling 720 fits

AdaBoost Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.957346  0.804293 0.721749   0.938981 0.990977
Best hyperparameters found by GridSearchCV:
{'algorithm': 'SAMME', 'estimator': DecisionTreeClassifier(max_depth=3), 'learning_rate': 1, 'n_estimators': 200}

Running AdaBoost with Normalized Data configuration...
Fitting 10 folds for each of 72 candidates, totalling 720 fits

AdaBoost Model Performance Metrics
 Dataset  Ac

---

# XGBoost

### XGBoost with PCA 90

In [22]:
# Written by Ovi, 2025-07-07, XGBoost classification with preprocessing and result logging



# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), 
                            X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with XGBoost ===")
xgb_estimator = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

rfecv = RFECV(estimator=xgb_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=xgb_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.90
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: XGBoost + GridSearchCV
print("\n=== XGBoost Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning XGBoost with {name} configuration...")
    xgb = GridSearchCV(
        XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    xgb.fit(X_train_cfg, y_train_cfg)

    y_train_xgb = xgb.predict(X_train_cfg)
    y_test_xgb = xgb.predict(X_test_cfg)
    y_train_xgb_proba = xgb.predict_proba(X_train_cfg)
    y_test_xgb_proba = xgb.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_xgb),
            metrics.accuracy_score(y_test, y_test_xgb),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.f1_score(y_test, y_test_xgb, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.recall_score(y_test, y_test_xgb, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.precision_score(y_test, y_test_xgb, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_xgb_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_xgb_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nXGBoost Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_xgb_proba, multi_class='ovr', average='macro')
    storeResults(
        'XGBoost 90',
        name,
        metrics.accuracy_score(y_test, y_test_xgb),
        metrics.f1_score(y_test, y_test_xgb, average='macro'),
        metrics.recall_score(y_test, y_test_xgb, average='macro'),
        metrics.precision_score(y_test, y_test_xgb, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(xgb.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 15

=== RFECV Feature Selection with XGBoost ===
Optimal number of features selected by RFECV: 15

=== PCA Dimensionality Reduction ===
Number of components that explain 90.0% variance: 13

=== XGBoost Model Performance with Hyperparameter Tuning ===

Running XGBoost with Original Data configuration...
Fitting 10 folds for each of 162 candidates, totalling 1620 fits

XGBoost Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.952607  0.755429 0.651515   0.983498 0.986044
Best hyperparameters found by GridSearchCV:
{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.6}

Running XGBoost with Normalized Data configuration...
Fitting 10 folds for each of 162 candidates, totalling 1620 fits

XGBoost Model Performance Metrics
 Dataset  Accuracy  F1 Scor

### XGBoost with PCA 95

In [23]:
# Written by Ovi, 2025-07-07, XGBoost classification with preprocessing and result logging


# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), 
                            X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with XGBoost ===")
xgb_estimator = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

rfecv = RFECV(estimator=xgb_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=xgb_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.95
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: XGBoost + GridSearchCV
print("\n=== XGBoost Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning XGBoost with {name} configuration...")
    xgb = GridSearchCV(
        XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    xgb.fit(X_train_cfg, y_train_cfg)

    y_train_xgb = xgb.predict(X_train_cfg)
    y_test_xgb = xgb.predict(X_test_cfg)
    y_train_xgb_proba = xgb.predict_proba(X_train_cfg)
    y_test_xgb_proba = xgb.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_xgb),
            metrics.accuracy_score(y_test, y_test_xgb),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.f1_score(y_test, y_test_xgb, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.recall_score(y_test, y_test_xgb, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.precision_score(y_test, y_test_xgb, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_xgb_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_xgb_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nXGBoost Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_xgb_proba, multi_class='ovr', average='macro')
    storeResults(
        'XGBoost 95',
        name,
        metrics.accuracy_score(y_test, y_test_xgb),
        metrics.f1_score(y_test, y_test_xgb, average='macro'),
        metrics.recall_score(y_test, y_test_xgb, average='macro'),
        metrics.precision_score(y_test, y_test_xgb, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(xgb.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 15

=== RFECV Feature Selection with XGBoost ===
Optimal number of features selected by RFECV: 15

=== PCA Dimensionality Reduction ===
Number of components that explain 95.0% variance: 14

=== XGBoost Model Performance with Hyperparameter Tuning ===

Running XGBoost with Original Data configuration...
Fitting 10 folds for each of 162 candidates, totalling 1620 fits

XGBoost Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.952607  0.755429 0.651515   0.983498 0.986044
Best hyperparameters found by GridSearchCV:
{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.6}

Running XGBoost with Normalized Data configuration...
Fitting 10 folds for each of 162 candidates, totalling 1620 fits

XGBoost Model Performance Metrics
 Dataset  Accuracy  F1 Scor

### XGBoost with PCA 99

In [24]:
# Written by Ovi, 2025-07-07, XGBoost classification with preprocessing and result logging



# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), 
                            X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with XGBoost ===")
xgb_estimator = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

rfecv = RFECV(estimator=xgb_estimator, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X_train_kbest, y_train)

print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=xgb_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.99
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: XGBoost + GridSearchCV
print("\n=== XGBoost Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning XGBoost with {name} configuration...")
    xgb = GridSearchCV(
        XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    xgb.fit(X_train_cfg, y_train_cfg)

    y_train_xgb = xgb.predict(X_train_cfg)
    y_test_xgb = xgb.predict(X_test_cfg)
    y_train_xgb_proba = xgb.predict_proba(X_train_cfg)
    y_test_xgb_proba = xgb.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_xgb),
            metrics.accuracy_score(y_test, y_test_xgb),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.f1_score(y_test, y_test_xgb, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.recall_score(y_test, y_test_xgb, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_xgb, average='macro'),
            metrics.precision_score(y_test, y_test_xgb, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_xgb_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_xgb_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nXGBoost Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_xgb_proba, multi_class='ovr', average='macro')
    storeResults(
        'XGBoost 99',
        name,
        metrics.accuracy_score(y_test, y_test_xgb),
        metrics.f1_score(y_test, y_test_xgb, average='macro'),
        metrics.recall_score(y_test, y_test_xgb, average='macro'),
        metrics.precision_score(y_test, y_test_xgb, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(xgb.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 15

=== RFECV Feature Selection with XGBoost ===
Optimal number of features selected by RFECV: 15

=== PCA Dimensionality Reduction ===
Number of components that explain 99.0% variance: 15

=== XGBoost Model Performance with Hyperparameter Tuning ===

Running XGBoost with Original Data configuration...
Fitting 10 folds for each of 162 candidates, totalling 1620 fits

XGBoost Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.952607  0.755429 0.651515   0.983498 0.986044
Best hyperparameters found by GridSearchCV:
{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.6}

Running XGBoost with Normalized Data configuration...
Fitting 10 folds for each of 162 candidates, totalling 1620 fits

XGBoost Model Performance Metrics
 Dataset  Accuracy  F1 Scor

---

# Bagging

### Bagging classification with PCA 90

In [26]:
# Written by Ovi, 2025-07-07, Bagging classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(BaggingClassifier(estimator=DecisionTreeClassifier()), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with Bagging ===")
# Use single DecisionTreeClassifier for RFECV to enable feature_importances_
tree_estimator = DecisionTreeClassifier(random_state=42)

rfecv = RFECV(
    estimator=tree_estimator,
    step=1,
    cv=StratifiedKFold(5),
    scoring='accuracy',
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=tree_estimator, n_features_to_select=rfecv.n_features_)

X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.90
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: BaggingClassifier + GridSearchCV
print("\n=== Bagging Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_samples': [0.6, 0.8, 1.0],
    'max_features': [0.6, 0.8, 1.0],
    'bootstrap': [True],
    'bootstrap_features': [False],
    'estimator': [ 
        DecisionTreeClassifier(max_depth=3, min_samples_split=2),
        DecisionTreeClassifier(max_depth=5, min_samples_split=5),
        DecisionTreeClassifier(max_depth=None, min_samples_split=10)
    ]
}


for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Bagging with {name} configuration...")
    bag = GridSearchCV(
        BaggingClassifier(),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    bag.fit(X_train_cfg, y_train_cfg)

    y_train_bag = bag.predict(X_train_cfg)
    y_test_bag = bag.predict(X_test_cfg)
    y_train_bag_proba = bag.predict_proba(X_train_cfg)
    y_test_bag_proba = bag.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_bag),
            metrics.accuracy_score(y_test, y_test_bag),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.f1_score(y_test, y_test_bag, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.recall_score(y_test, y_test_bag, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.precision_score(y_test, y_test_bag, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_bag_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_bag_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nBagging Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_bag_proba, multi_class='ovr', average='macro')
    storeResults(
        'Bagging 90',
        name,
        metrics.accuracy_score(y_test, y_test_bag),
        metrics.f1_score(y_test, y_test_bag, average='macro'),
        metrics.recall_score(y_test, y_test_bag, average='macro'),
        metrics.precision_score(y_test, y_test_bag, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(bag.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 22

=== RFECV Feature Selection with Bagging ===
Optimal number of features selected by RFECV: 1

=== PCA Dimensionality Reduction ===
Number of components that explain 90.0% variance: 1

=== Bagging Model Performance with Hyperparameter Tuning ===

Running Bagging with Original Data configuration...
Fitting 10 folds for each of 81 candidates, totalling 810 fits

Bagging Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.981013  0.916868 0.857639   0.993197 0.999923
    Test  0.933649  0.599386 0.530303   0.977346 0.954901
Best hyperparameters found by GridSearchCV:
{'bootstrap': True, 'bootstrap_features': False, 'estimator': DecisionTreeClassifier(min_samples_split=10), 'max_features': 0.8, 'max_samples': 1.0, 'n_estimators': 100}

Running Bagging with Normalized Data configuration...
Fitting 10 folds for each of 81 candidates, totalling 810 fits

### Bagging classification with PCA 95

In [27]:
# Written by Ovi, 2025-07-07, Bagging classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(BaggingClassifier(estimator=DecisionTreeClassifier()), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with Bagging ===")
# Use single DecisionTreeClassifier for RFECV to enable feature_importances_
tree_estimator = DecisionTreeClassifier(random_state=42)

rfecv = RFECV(
    estimator=tree_estimator,
    step=1,
    cv=StratifiedKFold(5),
    scoring='accuracy',
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=tree_estimator, n_features_to_select=rfecv.n_features_)

X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.95
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: BaggingClassifier + GridSearchCV
print("\n=== Bagging Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_samples': [0.6, 0.8, 1.0],
    'max_features': [0.6, 0.8, 1.0],
    'bootstrap': [True],
    'bootstrap_features': [False],
    'estimator': [ 
        DecisionTreeClassifier(max_depth=3, min_samples_split=2),
        DecisionTreeClassifier(max_depth=5, min_samples_split=5),
        DecisionTreeClassifier(max_depth=None, min_samples_split=10)
    ]
}


for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Bagging with {name} configuration...")
    bag = GridSearchCV(
        BaggingClassifier(),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    bag.fit(X_train_cfg, y_train_cfg)

    y_train_bag = bag.predict(X_train_cfg)
    y_test_bag = bag.predict(X_test_cfg)
    y_train_bag_proba = bag.predict_proba(X_train_cfg)
    y_test_bag_proba = bag.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_bag),
            metrics.accuracy_score(y_test, y_test_bag),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.f1_score(y_test, y_test_bag, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.recall_score(y_test, y_test_bag, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.precision_score(y_test, y_test_bag, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_bag_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_bag_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nBagging Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_bag_proba, multi_class='ovr', average='macro')
    storeResults(
        'Bagging 95',
        name,
        metrics.accuracy_score(y_test, y_test_bag),
        metrics.f1_score(y_test, y_test_bag, average='macro'),
        metrics.recall_score(y_test, y_test_bag, average='macro'),
        metrics.precision_score(y_test, y_test_bag, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(bag.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 10

=== RFECV Feature Selection with Bagging ===
Optimal number of features selected by RFECV: 2

=== PCA Dimensionality Reduction ===
Number of components that explain 95.0% variance: 2

=== Bagging Model Performance with Hyperparameter Tuning ===

Running Bagging with Original Data configuration...
Fitting 10 folds for each of 81 candidates, totalling 810 fits

Bagging Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.958861  0.794992 0.704861   0.985604 0.998462
    Test  0.933649  0.599386 0.530303   0.977346 0.967648
Best hyperparameters found by GridSearchCV:
{'bootstrap': True, 'bootstrap_features': False, 'estimator': DecisionTreeClassifier(max_depth=5, min_samples_split=5), 'max_features': 0.6, 'max_samples': 1.0, 'n_estimators': 150}

Running Bagging with Normalized Data configuration...
Fitting 10 folds for each of 81 candidates, totall

### Bagging classification with PCA 99

In [28]:
# Written by Ovi, 2025-07-07, Bagging classification with preprocessing and result logging

# Store different configurations
configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 2: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# Step 3.1: SelectKBest
print("\n=== SelectKBest Feature Selection ===")
scores = []
for k in range(1, X_train.shape[1] + 1):
    kbest = SelectKBest(score_func=f_classif, k=k)
    X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
    score = cross_val_score(BaggingClassifier(estimator=DecisionTreeClassifier()), X_train_kbest, y_train, cv=5, scoring='accuracy').mean()
    scores.append(score)

optimal_k = scores.index(max(scores)) + 1
print(f"Optimal number of features to select using SelectKBest: {optimal_k}")

kbest = SelectKBest(score_func=f_classif, k=optimal_k)
X_train_kbest = kbest.fit_transform(X_train_normalized, y_train)
X_test_kbest = kbest.transform(X_test_normalized)
selected_features_kbest = X.columns[kbest.get_support()]
configurations.append(('SelectKBest', X_train_kbest, X_test_kbest, y_train))

# Step 3.2: RFECV
print("\n=== RFECV Feature Selection with Bagging ===")
# Use single DecisionTreeClassifier for RFECV to enable feature_importances_
tree_estimator = DecisionTreeClassifier(random_state=42)

rfecv = RFECV(
    estimator=tree_estimator,
    step=1,
    cv=StratifiedKFold(5),
    scoring='accuracy',
    n_jobs=-1
)
rfecv.fit(X_train_kbest, y_train)
print(f"Optimal number of features selected by RFECV: {rfecv.n_features_}")

rfe = RFE(estimator=tree_estimator, n_features_to_select=rfecv.n_features_)
X_train_rfe = rfe.fit_transform(X_train_kbest, y_train)
X_test_rfe = rfe.transform(X_test_kbest)
selected_features_rfe = selected_features_kbest[rfe.get_support()]
configurations.append(('RFECV', X_train_rfe, X_test_rfe, y_train))

# Step 3.3: PCA
print("\n=== PCA Dimensionality Reduction ===")
pca = PCA().fit(X_train_rfe)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
desired_variance = 0.99
n_components = np.argmax(cumulative_variance >= desired_variance) + 1
print(f'Number of components that explain {desired_variance * 100}% variance: {n_components}')

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_rfe)
X_test_pca = pca.transform(X_test_rfe)
configurations.append(('PCA', X_train_pca, X_test_pca, y_train))

# Step 4: BaggingClassifier + GridSearchCV
print("\n=== Bagging Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_samples': [0.6, 0.8, 1.0],
    'max_features': [0.6, 0.8, 1.0],
    'bootstrap': [True],
    'bootstrap_features': [False],
    'estimator': [ 
        DecisionTreeClassifier(max_depth=3, min_samples_split=2),
        DecisionTreeClassifier(max_depth=5, min_samples_split=5),
        DecisionTreeClassifier(max_depth=None, min_samples_split=10)
    ]
}


for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Bagging with {name} configuration...")
    bag = GridSearchCV(
        BaggingClassifier(),
        param_grid,
        cv=10,
        n_jobs=-1,
        verbose=2
    )
    bag.fit(X_train_cfg, y_train_cfg)

    y_train_bag = bag.predict(X_train_cfg)
    y_test_bag = bag.predict(X_test_cfg)
    y_train_bag_proba = bag.predict_proba(X_train_cfg)
    y_test_bag_proba = bag.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_bag),
            metrics.accuracy_score(y_test, y_test_bag),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.f1_score(y_test, y_test_bag, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.recall_score(y_test, y_test_bag, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_bag, average='macro'),
            metrics.precision_score(y_test, y_test_bag, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_bag_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_bag_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nBagging Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_bag_proba, multi_class='ovr', average='macro')
    storeResults(
        'Bagging 99',
        name,
        metrics.accuracy_score(y_test, y_test_bag),
        metrics.f1_score(y_test, y_test_bag, average='macro'),
        metrics.recall_score(y_test, y_test_bag, average='macro'),
        metrics.precision_score(y_test, y_test_bag, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(bag.best_params_)



=== SelectKBest Feature Selection ===
Optimal number of features to select using SelectKBest: 13

=== RFECV Feature Selection with Bagging ===
Optimal number of features selected by RFECV: 1

=== PCA Dimensionality Reduction ===
Number of components that explain 99.0% variance: 1

=== Bagging Model Performance with Hyperparameter Tuning ===

Running Bagging with Original Data configuration...
Fitting 10 folds for each of 81 candidates, totalling 810 fits

Bagging Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.957278  0.783941 0.694444   0.985075 0.998444
    Test  0.928910  0.558176 0.488636   0.975845 0.927392
Best hyperparameters found by GridSearchCV:
{'bootstrap': True, 'bootstrap_features': False, 'estimator': DecisionTreeClassifier(max_depth=5, min_samples_split=5), 'max_features': 1.0, 'max_samples': 0.6, 'n_estimators': 100}

Running Bagging with Normalized Data configuration...
Fitting 10 folds for each of 81 candidates, totall

---

# Results

In [29]:
# Creating the dataframe
result = pd.DataFrame({
    'ML Model': ML_Model,
    'Configuration': ML_Config,
    'Accuracy': [f"{acc * 100:.3f}%" for acc in accuracy],
    'F1 Score': [f"{f1 * 100:.3f}%" for f1 in f1_score],
    'Recall': [f"{rec * 100:.3f}%" for rec in recall],
    'Precision': [f"{prec * 100:.3f}%" for prec in precision],
    'ROC_AUC': [f"{roc * 100:.3f}%" for roc in auc_roc],
})

# Remove duplicates based on model and configuration
result.drop_duplicates(subset=["ML Model", "Configuration"], inplace=True)

# Display the result
print("\n" + "=" * 100)
print("MODEL PERFORMANCE RESULTS")
print("=" * 100)
print(result.to_string(index=False))

# Save the result to a CSV file
result.to_csv('results/model_results.csv', index=False)
print("\nResults saved to model_results.csv")

# Sort by Accuracy and F1 Score
sorted_result = result.sort_values(by=['F1 Score', 'Accuracy'], ascending=False).reset_index(drop=True)

# Display the sorted result
print("\n" + "=" * 100)
print("SORTED MODEL PERFORMANCE RESULTS (by Accuracy and F1 Score)")
print("=" * 100)
print(sorted_result.to_string(index=False))

# Save the sorted result
sorted_result.to_csv('results/sorted_model_results.csv', index=False)
print("\nSorted results saved to sorted_model_results.csv")

# Extract top configuration per ML model
top_per_model = sorted_result.groupby('ML Model', as_index=False).first()

# Display and save the top configuration table
print("\n" + "=" * 100)
print("TOP CONFIGURATION PER MODEL")
print("=" * 100)
print(top_per_model.to_string(index=False))

top_per_model.to_csv('results/top_configurations.csv', index=False)
print("\nTop configuration per model saved to top_configurations.csv")



MODEL PERFORMANCE RESULTS
                 ML Model   Configuration Accuracy F1 Score  Recall Precision ROC_AUC
Support Vector Machine 90   Original Data  98.104%  92.171% 86.742%   99.320% 99.754%
Support Vector Machine 90 Normalized Data  98.104%  92.171% 86.742%   99.320% 99.692%
Support Vector Machine 90     SelectKBest  98.104%  92.171% 86.742%   99.320% 99.664%
Support Vector Machine 90           RFECV  98.104%  92.171% 86.742%   99.320% 99.646%
Support Vector Machine 90             PCA  99.052%  96.494% 93.939%   99.656% 99.982%
Support Vector Machine 95   Original Data  98.104%  92.171% 86.742%   99.320% 99.745%
Support Vector Machine 95 Normalized Data  98.104%  92.171% 86.742%   99.320% 99.710%
Support Vector Machine 95     SelectKBest  98.104%  92.171% 86.742%   99.320% 99.646%
Support Vector Machine 95           RFECV  98.104%  92.171% 86.742%   99.320% 99.646%
Support Vector Machine 95             PCA  98.578%  94.186% 89.773%   99.487% 99.644%
Support Vector Machine 99  

In [30]:
import pandas as pd

# Read input CSV
df = pd.read_csv('results/top_configurations.csv')

# Sort by 'Accuracy' column in descending order
df_sorted = df.sort_values(by=['F1 Score', 'Accuracy'], ascending=False)

# Save the sorted DataFrame to a new CSV
df_sorted.to_csv('results/sorted_top_configurations.csv', index=False)

---

# END